In [1]:
%load_ext autoreload
%autoreload 2
import functools
import json
import urllib.request
import urllib.parse
import urllib.error
import subprocess
import json
import base64
from hashlib import sha256
import fonction
import fonction_utile
import time
import sympy
from random import *
from binascii import unhexlify, hexlify
from math import log

In [2]:
class ServerError(Exception):
    """
    Exception déclenchée en cas de problème côté serveur (URL incorrecte,
    accès interdit, requête mal formée, etc.)
    """
    def __init__(self, code=None, msg=None):
        self.code = code
        self.msg = msg

    def __str__(self):
        return "ERREUR {}, {}".format(self.code, self.msg)

In [3]:
class Connection:
    """
    Cette classe sert à ouvrir et à maintenir une connection avec le système
    UGLIX. Voir les exemples ci-dessous.

    Pour créer une instance de la classe, il faut spécifier une ``adresse de 
    base''. Les requêtes se font à partir de là, ce qui est bien pratique.
    L'adresse de base est typiquement l'adresse du système UGLIX.

    Cet objet Connection() s'utilise surtout via ses méthodes get(), post()...

    Il est conçu pour pouvoir être étendu facilement. En dériver une sous-classe
    capable de gérer des connexions chiffrées ne nécessite que 20 lignes de
    code supplémentaires.subject

    Exemple :
    >>> c = Connection()
    >>> c.get('/bin/echo')
    'usage: echo [arguments]'

    >>> print(c.get('/'))   # doctest: +ELLIPSIS
    HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
    ...
    """
    def __init__(self, base_url="http://isec.fil.cool/uglix"):
        self._base = base_url
        self._session = None   # au départ nous n'avons pas de cookie de session

    ############################################################################
    #                          MÉTHODES PUBLIQUES                              #
    ############################################################################
    def get(self, url):
        """
        Charge l'url demandée. Une requête HTTP GET est envoyée.

        >>> c = Connection()
        >>> c.get('/bin/echo')
        'usage: echo [arguments]'

        En cas d'erreur côté serveur, on récupère une exception.
        >>> c.get('/bin/foobar') # doctest: +ELLIPSIS
        Traceback (most recent call last):
        ...
        client.ServerError: ERREUR 404, ...
        """
        # prépare la requête
        request = urllib.request.Request(self._base + url, method='GET')
        return self._query(url, request)


    def post(self, url, **kwds):
        """
        Charge l'URL demandée. Une requête HTTP POST est envoyée. Il est 
        possible d'envoyer un nombre arbitraire d'arguments supplémentaires
        sous la forme de paires clef-valeur. Ces paires sont encodées sous la
        forme d'un dictionnaire JSON qui constitue le corps de la requête.

        Python permet de spécifier ces paires clef-valeurs comme des arguments
        nommés de la méthode post(). On peut envoyer des valeurs de n'importe
        quel type sérialisable en JSON.

        Par exemple, pour envoyer un paramètre nommé "string_example" de valeur
        "toto et un paramètre nommé "list_example" de valeur [True, 42, {'foo': 'bar'}],
        il faut invoquer :

        >>> c = Connection()
        >>> c.post('/bin/echo', string_example="toto", list_example=[True, 42, {'foo': 'bar'}])
        {'content_found': {'string_example': 'toto', 'list_example': [True, 42, {'foo': 'bar'}]}}

        L'idée est que la méthode post() convertit ceci en un dictionnaire JSON,
        qui ici ressemblerait à :

        {'string_example': 'toto', 'list_example': [True, 42, {'foo': 'bar'}]},

        puis l'envoie au serveur.
        """
        # prépare la requête
        request = urllib.request.Request(self._base + url, method='POST')
        data = None
        # kwds est un dictionnaire qui contient les arguments nommés. S'il
        # n'est pas vide, on l'encode en JSON et on l'ajoute au corps de la
        # requête.
        if kwds:     
            request.add_header('Content-type', 'application/json')
            data = json.dumps(kwds).encode()
        return self._query(url, request, data)


    def put(self, url, content):
        """
        Charge l'URL demandée avec une requête HTTP PUT. L'argument content
        forme le corps de la requête. Si content est de type str(), il est
        automatiquement encodé en UTF-8. cf /doc/strings pour plus de détails
        sur la question.
        """
        request = urllib.request.Request(self._base + url, method='PUT')
        if isinstance(content, str):
            content = content.encode()
        return self._query(url, request, data=content)

    ############################################################################
    #                     MÉTHODES PUBLIQUES AVANCÉES                          #
    ############################################################################


    def post_raw(self, url, data, content_type='application/octet-stream'):
        """
        Charge l'url demandée avec une requête HTTP POST. L'argument data
        forme le corps de la requête. Il doit s'agir d'un objet de type 
        bytes(). Cette méthode est d'un usage plus rare, et sert à envoyer des
        données qui n'ont pas vocation à être serialisées en JSON (comme des
        données binaires chiffrées, par exemple).

        Principalement utilisé pour étendre le client et lui ajouter des
        fonctionnalités.
        """
        request = urllib.request.Request(self._base + url, method='POST')
        request.add_header('Content-type', content_type)
        return self._query(url, request, data)

    def close_session(self):
        """
        Oublie la session actuelle. En principe, personne n'a besoin de ceci.
        """
        self._session = None


    ############################################################################
    #                          MÉTHODES INTERNES                               #
    ############################################################################

    def _pre_process(self, request):
        """
        Effectue un pré-traitement sur la requête pas encore lancée.
        Si on possède un cookie de session, on l'injecte dans les en-tête HTTP.
        """
        request.add_header('User-Agent', 'UGLIX official client v2.1 (c) sysprog')
        if self._session:
            request.add_header('Cookie', self._session)

    def _post_process(self, result, http_headers):
        """
        Effectue un post-traitement sur le résultat "brut" de la requête. En
        particulier, on décode les dictionnaires JSON, et on convertit le texte
        (encodé en UTF-8) en chaine de charactère Unicode. On peut étendre cette
        méthode pour gérer d'autres types de contenu si besoin.
        """
        if 'Content-Type' in http_headers:
            if http_headers['Content-Type'] == "application/json":
                return json.loads(result.decode())
            if http_headers['Content-Type'].startswith("text/plain"):
                return result.decode()
        # on ne sait pas ce que c'est : on laisse tel quel
        return result

    def _query(self, url, request, data=None):
        """
        Cette fonction à usage interne est appelée par get(), post(), put(),
        etc. Elle reçoit en argument une url et un objet Request() du module
        standard urllib.request.
        """
        self._pre_process(request)
        try:           
            # lance la requête. Si data n'est pas None, la requête aura un
            # corps non-vide, avec data dedans.
            with urllib.request.urlopen(request, data) as connexion:
                # récupère les en-têtes HTTP et le corps de la réponse, puis
                # ferme la connection
                headers = dict(connexion.info())
                result = connexion.read()
            
            # si on reçoit un identifiant de session, on le stocke
            if 'Set-Cookie' in headers:
                self._session = headers['Set-Cookie']

            # on effectue le post-processing, puis on renvoie les données.
            # c'est fini.
            return self._post_process(result, headers)

        except urllib.error.HTTPError as e:
            # On arrive ici si le serveur a renvoyé un code d'erreur HTTP
            # (genre 400, 403, 404, etc.). On récupère le corps de la réponse
            # car il y a peut-être des explications dedans. On a besoin des
            # en-tête pour le post-processing.
            headers = dict(e.headers)
            message = e.read()
            raise ServerError(e.code, self._post_process(message, headers)) from None



In [6]:

# ce script suppose qu'il a affaire à OpenSSL v1.1.1
# vérifier avec "openssl version" en cas de doute.
# attention à MacOS, qui fournit à la place LibreSSL.

# en cas de problème, cette exception est déclenchée
class OpensslError(Exception):
    pass

# Il vaut mieux être conscient de la différence entre str() et bytes()
# cf /usr/doc/strings.txt


def encrypt(plaintext, passphrase, cipher='aes-128-cbc'):
    """invoke the OpenSSL library (though the openssl executable which must be
       present on your system) to encrypt content using a symmetric cipher.

       The passphrase is an str object (a unicode string)
       The plaintext is str() or bytes()
       The output is bytes()

       # encryption use
       >>> message = "texte avec caractères accentués"
       >>> c = encrypt(message, 'foobar')
       
    """
    # prépare les arguments à envoyer à openssl
    pass_arg = 'pass:{0}'.format(passphrase)
    args = ['openssl', 'enc', '-' + cipher, '-base64', '-pass', pass_arg, '-pbkdf2']
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=plaintext, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [4]:
# vérifie l'authenticité de la signature du serveur
from zlib import decompress as Y
from base64 import b85decode as Z

public_key = 'V<bQ*EigZNBrR`4RU|-cFmOX>Fiv1-H&bLOcX>@%Mrm+jR$*2}ZA?f`cxgdjF?C' \
             'jWPB$YzDtbyiPftxZP&ipPDtlKpJ$NuyUR7&Ta4<YUdRTmEVKQ}TRB$9VC2=c6V' \
             'OLL1ZZSJ=D^O#7PE#u(ZC+SvUwm~pH#0DHVmwkkSVlKOO;~ABPJLoiS|mY9YIkv' \
             '7Jzh+Cb6zbbBT0EyMrnC|Y-&+@cQR#nMsjyaX?AN(awJAHNhmfWM_5!zEh=O+Jy' \
             'J$1M|yQoKXFcFMPFlDU~V@;Vof|NGb=neLpN_XZEs>(W+ZtuEmJ=<U|=aVC@pG0' \
             'G$cYdX(3^LZDDD1Pe(XWM@?^VYGP7vPD5y6A$?bRWlbSPQ$$K`T1ZwaJ}X#uVmN' \
             '9tW+pUsQZ#ZQM}9RzVl{esR#h}WR5L<7WnXMIQ(k3gST|rhLw0dtMl&!pMsY<cH' \
             '&a+sNLX_zdMGkEWOy_}cs^+(C^%6wa5PhRMkZ-KMqX%1K2<VmLR5GtD?Da$U?VA' \
             'bMKC!iHdkI>O=LeXUSm~RF-Rm)HDqB`bty}GXmxryZc<ZYKxtojKuIu6S8HZOXF' \
             'ygtV0SAmd3sDHJ49tBZ!vXLa8e_5UsG;<QbSp1R%cLTP%(RNG%{dfPh=z}J4j=9' \
             'J#IB@He+UJC1F!dGJ8H$SwJ~%STkvES#WM-C^J%IdSW&=B`97yVQ^JTb15o%U~y' \
             'JSXni4LV?tAQd|o6dR(NS)BUmL=Nj_CIB}g+uR%}CRPIhi|KYUMOHDY`|UQshtH' \
             'EeNKKru*AAy+jeHFPmjPkd2ia7bPuY$Q))XmD_Abwyw+Z9OY0Iaqa7CTcuob~Hb' \
             'CCN_6^RY74?cuz1XC`3n5es(@lFeX_yFf~?Dc0+G_PfKo9HcWj!P$XJAOFnUJJ8' \
             'msWdqjD2NP2t'
seed       = 'V<cc<Lr!ckd}m@*XfSXpFibpfKu<74MMg18D|9JAV{~ptbWCJbVLT>9Y;1HsH)j'
algorithm  = 'c%1E6%?^Sv41Sdz+gI32aAjT$$PPw~Mx$Xoy>3opbO`%VssahDw3N2rSDFm!VqI' \
             '1R%Q&;EoHGyrkY7*KN2lzNUK^+JY=rHhte1=VN-A&1kIK-XjgUpY4~fccguRs2T' \
             'QL~wdywrXFblvpq3h@Jr38m$;61jZ=P|rBT{S?MA&CA6iKI`GA>=4AQTy^tXm)=' \
             'KwLK;ioybCf>Bq#0CW1T`wNGjj++^U6f=v8Mb#y5Li0}W2aJ)kLk_t2=XDm6oCU' \
             'QP{9#KCf(<o$!`#-Zl*t^}HX>J=einE4oc1+#t8mFCIwOZdIsPm=EBjO3eu;k+'
signature = eval(Y(Z(algorithm)))
if not signature.startswith(b'S50|UU'):
    raise ValueError("ATTENTION : le serveur a été hacké !")
    



In [1]:
login = "palmerjohn"
password = "j^5vL+Fpv@"
c = Connection()
def chapConnection(c, username,password): 
    challenge = c.get('/bin/login/CHAP')
    challenge = challenge['challenge']
    cipher = encrypt(username + '-' + challenge, password)
    return c.post('/bin/login/CHAP', user = username, response = cipher)
chapConnection(c, login, password)

NameError: name 'Connection' is not defined

In [9]:
def s2file(string,filename, mode = 'w'):
    with open(filename, mode) as f:
        f.write(string) 

In [10]:
def mail(username, numero = "", body = ""): 
    commande = '/home/' + username + '/INBOX'
    if numero != "": 
        num_mail = "/" + numero 
        commande = commande + num_mail + body
    print(c.get(commande))

In [13]:
c.get('/bin/echo')

'usage: echo [arguments]'

In [15]:
c.post('/bin/echo', string_example="toto", list_example=[True, 42, {'foo': 'bar'}])

{'content_found': {'string_example': 'toto',
  'list_example': [True, 42, {'foo': 'bar'}]},
 '[[kernel-monitord[42] UGLIX]]': 'ECHO:14:1@533419|98db887529bdd6cb25fbe141efdf7306'}

In [30]:
c.get('/bin/login')

'USAGE: /bin/login [user] [password]'

In [9]:
c.post('/bin/login', user= 'guest', password= 'guest')

'PAP login successful. Access to /home/guest granted.\n[ You have new mail ]'

In [10]:
c.get('/home/guest/INBOX/15')

'----------------------------------------------------------------------\nMessage #15.\n\n\nDate: 03-01-21893 18:53:25\nX-UGLIX-Spam: [[kernel-spamd[37] UGLIX]] GUEST.MAIL:14:1@955297|2a0407cdc9cc0de29c24974525425816\nFrom: bwoolley@stocker-verlag.com\nSubject: Do you desire to gratify your girlfriend tonight?\n\nhttp://eagerness.zxkaufvq.ru/\n\n\n'

In [11]:
c.get('/home/guest')

'# Bienvenue, guest.\n# Dernière connection le 06-02-21893 20:07:27 (protocole : PAP)\n# Vous avez 21 messages non lus (2368 en tout).\n\n================================================================================\nBienvenue sur le compte invité. Il s\'agit s\'un service offert à la communauté.\n\nNous avons récemment observé un nombre grandissant d\'abus dans l\'usage de ce \ncompte. Par exemple, le compte invité n\'est pas censé servir de dépôt de \nfichiers chiffrés (ceci contrevient à une disposition de la règlementation \nCRX3-42b). Le fichier ``NASA.bin\'\' n\'a aucune raison d\'être là.\n\nDe plus, le mot de passe ("ISEC") est tellement faible que nous avons pu le \ncracker en quelques instants. Ce genre de chiffrement low-cost n\'offre en \nréalité aucune protection. Cf. /doc/encryption pour plus de détails.\n---\nroot\n\n[[1612642047##kernel-monitord[42] UGLIX]]: GUEST.MOTD:14:1@713701|20f839dabbcbb1e19bcacbf8029d873b\n=====================================================

In [12]:
c.get('/home/guest/NASA.bin')

'U2FsdGVkX1+50ehSUSFtABFsoGXe3ep+wcW1leSVK3MvbthH1fIS5qifSoQr28u0\n5k+7r9Sh2qfkfPL+Ml1D4KbZ0fvRcKLRV6EPDetRCO3m6TzZa1nvJJmFqiv58DPd\ndlxOiScfSR7xkXXUhPJgae5R32mK+/Vjwf2X4Nkrqos6hmP6uIb45ArVpWtWSXSE\nyVDd5xjd37nmFk2fLyiWBzJfLGOQXJgEkcVre79/JDfC97t3DCkBeuYgqmektDNB\nMFPDDfFD/0hTz9Y9r2r9x/LGWG7OT7eozv+JzNZFOahSgOx/IxrFU33tKAQjmllj\ntql/ElAAUCUej3+OizjEAlHb+pawgImAsKeKwR8R2mWEorgyMdT2T6+qldmxwr+q\n+0bJ9UNtXn4UQxestbm7O7fyDbf58DexI6DOXHGHXqOW6chj/g2KFvePFs2miMGA\nhp0KoILb+mvXC6+fF+qP+UrawWrZuvuytfCuVeIgEHkzjnZShP7qjw6rkgfAZTzP\nYn4PlJTarbEmV4g/RZZmSQPxJmKew0MWLsvoQZU4lfKp7Wgk/sGlqseXoPQJCgNe\nwAdwEF5/b81qZTWKpnlitWCy5h8QJ/qOADt20T3bqCvJ4pdowakDg4IxMSQzWkb0\nDt0hY1Rf/b4o+9sbk5jxZipMxMrHmzHBT0PQ2T3fjuUxfVfapqYe7upNUmBrbVox\npGM3Rw0sxvOx9N4clNZ/Oto4knYJ2vCpSBHHlZ8L0UuDgIJSUb9O2ZFsyGRmnZLL\n4brCcXcDUkb9/1R7toOHmoxZTgwTQ51tapLKBzGSxyVp08Ot0FNT+SisCdm9IFZW\nPLNk/gNqnsbJO4VWg2+8cJDaQr92AwEZPhI/VxrGW/svFttxApA8I1GHIfhyyPYg\nlDNk/toyjwoWmlYaaRBc6reZdgm+kRaDsP2enDRjpgH7E9MQ2eC4bhJHwZ514fXP\njp9N+eV3J

In [13]:
c.get('/home/guest/README.txt')

'Les informations importantes sont contenues dans le fichier "NASA.bin", mais il\nest chiffré avec OpenSSL.\n\nLe mot de passe est "ISEC" (sans les guillemets, bien sûr).\n\nLisez "/doc/encryption" pour plus de détails sur la procédure de déchiffrement.\n\nBon courage,\n---\nLe manager'

In [6]:
def decrypt(file_name, password, cipher='aes-128-cbc'):
    # prépare les arguments à envoyer à openssl
    pass_arg = 'pass:{0}'.format(password)
    args = ['openssl', 'enc', '-d', '-base64', '-' + cipher, '-pbkdf2', '-pass', pass_arg, '-in', file_name]
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(file_name, str):
        file_name = file_name.encode('utf-8')
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=file_name, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [11]:
nasa = c.get('/home/guest/NASA.bin')
with open ('NASA.bin', 'w') as f:
    f.write(nasa)

ServerError: ERREUR 403, Permission denied.

In [17]:
resultat = decrypt('NASA.bin', 'ISEC')
print(resultat)

Après plusieurs années d'efforts, mon équipe est finalement parvenue à
obtenir accès à des comptes (autres que "guest") dans le système. Nous
avons découvert un fichier publiquement accessible contenant les
informations suivantes :



  nom         : Bailey Brown
  entreprise  : Obrien, Collins and Love
  login       : palmerjohn
  password    : j^5vL+Fpv@



Ce qui est très bizarre, c'est que ce fichier a disparu tout de suite
après que nous l'ayons lu ! Lors de la deuxième tentative d'accès, au
lieu d'accéder au contenu déjà présent, nous avons trouvé :

[[1612642049##kernel-monitord[42] UGLIX]]: GUEST.FILE:3030:1@320626|f6dc4e0aebb252d157bd7dc0a299ef4f


Au début nous avons cru qu'il s'agissait peut-être d'une bévue commise
par un utilisateur ou un "administrateur" peu précautionneux... Mais la
disparition du fichier, et le message ci-dessus laissent penser que
nous avons, involontairement, interagi avec quelqu'un ou quelque chose. 
Se pourrait-il ***qu'on*** nous ait aidé ?

En att

In [23]:
c.get('/bin/login')
c.post('/bin/login', user= 'palmerjohn', password= 'j^5vL+Fpv@')

'PAP login successful. Access to /home/palmerjohn granted.\n[ You have new mail ]'

In [24]:
c.post('/bin/jukebox/stop')

'Music stopped.'

In [25]:
print(c.get('/home/palmerjohn'))

# Bienvenue, palmerjohn.
# Dernière connection le 12-02-21893 11:59:06 (protocole : PAP)
# Vous avez 62 messages non lus (84 en tout).

Rumeur du jour: 

Travailler dans les datacenter pourrait être dangereux.

***** Contenu de votre dossier personnel ******

  /home/palmerjohn/bye-bye.txt



In [26]:
c.get('/home/palmerjohn/bye-bye.txt')

"Si vous lisez ces lignes, c'est que vous avez remarqué mon absence. Je \nm'explique en deux mots :\n\n*) J'en ai marre de ce p&£@%n de helpdesk de m#$§e. Les clients sont \n   insupportables et les managers sont pire encore.\n\n*) J'ai envie de VIVRE, d'avoir une VRAIE vie, de voir des VRAIS gens, de faire\n   le tour du VRAI monde, d'être VRAIMENT heureux et pas d'en avoir juste \n   l'impression.\n\nBref, je suis parti. Bon vent. Bonne chance pour me retrouver.  Et si ça \ndésorganise le helpdesk, eh bien... tant pis pour eux !\n\nà bon entendeur, salut."

In [13]:
c.get('/home/palmerjohn/INBOX')

"----------------------------------------------------------------------\nINBOX: There are more than 25 messages. Only displaying the 25 last messages.\n\nMessage | Read | Sender                   | Subject\n--------+------+--------------------------+---------------------------\n 11960  |   Y  | anonymous.coward         | Il faut qu'on parle\n 11959  |   N  | anonymous.coward         | Il faut qu'on parle\n 11958  |   N  | anonymous.coward         | Il faut qu'on parle\n 11957  |   N  | anonymous.coward         | Il faut qu'on parle\n 11954  |   Y  | CLEA Director            | La curiosité est un vilain défaut\n 11519  |   Y  | janice66                 | Re: Signature\n 11511  |   N  | janice66                 | Re: Signature\n 11504  |   N  | janice66                 | Re: Signature\n 11502  |   N  | janice66                 | Re: Signature\n 11499  |   N  | janice66                 | Re: Signature\n 11498  |   Y  | janice66                 | Re: Signature\n 11497  |   Y  | janice66   

In [27]:
c.get('/home/palmerjohn/INBOX/11180')

'----------------------------------------------------------------------\nMessage #11180.\n\n\nDate: 06-02-21893 20:08:24\nFrom: Help Desk Administrator\nSubject: [work] Vous avez 4202 tickets en attente\n\nCher(e) Bailey,\n\nVous avez 4202 tickets en attente. Je vous rappelle que notre\nentreprise, Obrien, Collins and Love, se fait un honneur d\'honorer\nles requêtes de ses clients en temps et en heure. En tant\nqu\'expert(e), vous avez un engagement moral à assurer l\'excellence du\ndépartement de S.A.V. cryptographique.\n\nJe vous prie donc de vous connecter au plus vite sur l\'interface de\ntraitement des tickets :\n\n    /bin/crypto_helpdesk\n\nAu cas où vous seriez une jeune recrue du service, je vous rappelle\nles grandes lignes de notre mission. Nos clients, lorsqu\'ils désirent\nfaire appel à nos service, ouvrent un "ticket" dans le système de\nsuivi. L\'administrateur du système de suivi vous affecte des ticket.\nIl vous incombe alors de répondre au problème de l\'utilisateur.

In [358]:
c.get('/bin/crypto_helpdesk')

Exception in thread Thread-59:
Traceback (most recent call last):
  File "/home/mohamed/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner


'[initializing ticket system]\n[connecting to tickets daemon]\n[fetching tickets]---> 6 tickets in queue\n[building UI]\n[startup complete]\n\nHello palmerjohn,\n\nWelcome to Obrien, Collins and Love Helpdesk User Interface.\n\nYour pending ticket list:\n-----------+-------+----------------------------------+------------------------------------\n|  id      | level | client                           | mission\n-----------+-------+----------------------------------+------------------------------------\n| 1124     | 1     | Nguyen, Cruz and Osborn          | Message à déchiffrer (clef publique)\n| 1125     | 3     | Nguyen, Cruz and Osborn          | Chiffrement hybride\n| XXXXXXXX | XXXXX | XXX SESSION SECURITY TOO LOW XXX | XXX PLEASE USE CHAP LOGIN (/doc/CHAP) XXX \n| XXXXXXXX | XXXXX | XXX SESSION SECURITY TOO LOW XXX | XXX PLEASE USE CHAP LOGIN (/doc/CHAP) XXX \n| XXXXXXXX | XXXXX | XXX SESSION SECURITY TOO LOW XXX | XXX PLEASE USE CHAP LOGIN (/doc/CHAP) XXX\n\nIf you need assistance

    self.run()
  File "/home/mohamed/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<string>", line 87, in _play
  File "<string>", line 1437, in __new__
  File "<string>", line 3771, in libvlc_new
  File "<string>", line 271, in _Cfunction
NameError: no function 'libvlc_new'


In [359]:
c.get('/bin/crypto_helpdesk/ticket/1111')

'Bonjour,\n\nJe suis Erika Mendez, le  gestionnaire du SAV\ncrypto. \n\nCe ticket-tutoriel a pour but de vous  guider lors des différentes étapes de \ntraitement d\'un ticket. Il n\'y a pas de vrai travail cryptographique à faire.\n\nIl n\'est pas obligatoire de le traiter pour passer à la suite.\n\nCe ticket est accompagné par une pièce jointe d\'identifiant "fetch-me". Pour la\nrécupérer, il vous faut donc lire le fichier ``virtuel\'\' : \n    \n    /bin/crypto_helpdesk/ticket/1111/attachment/fetch-me\n\nPour pouvoir fermer ce ticket, vous devez m\'envoyer un email, à l\'adresse \n``scottalyssa\'\'. Le corps de ce mail doit être un dictionnaire\ncontenant deux paires clef-valeur :\n\n  + \'foo\' : le contenu de la pièce jointe\n  + \'bar\' : le nombre 42\n\nNotez que tous les tickets ont une pièce-jointe spéciale nommée "client", que \nvous pouvez donc lire ici :\n\n    /bin/crypto_helpdesk/ticket/1111/attachment/client\n\nEt qui contient l\'adresse email de celui qui a ouvert le tic

In [360]:
c.get('/bin/crypto_helpdesk/ticket/1111/attachment/fetch-me')

'Operational researcher'

In [361]:
c.get('/bin/crypto_helpdesk/ticket/1111/attachment/client')

{'name': 'Erika Mendez',
 'company': 'Crypto Helpdesk Management',
 'email': 'scottalyssa'}

In [ ]:
c.post('/bin/sendmail', to='scottalyssa', subject="ticket", content={'foo':'Operational researcher', 'bar':42})

In [42]:
c.post('/bin/crypto_helpdesk/ticket/1111/close')

'STATUS : OK --- ticket 1111 closed\n\n[[1612642487##desk-trackerd[37] UGLIX]]: SAV.TUTO:3030:1@841824|18db9d9224690bd5744ef5027b60de09\n[ You have new mail ]'

In [362]:
c.get('/bin/crypto_helpdesk/ticket/1112')

'Bonjour,\n\nJe suis assistant de direction chez Boyer PLC, et j\'ai un \nproblème.\n\nMon N+1 insiste pour que je lui transmette rapidement une version chiffrée d\'un \nfichier qui se trouve sur un périphérique de stockage à long terme. Plus \nprécisément, il s\'agit du contenu de la bande magnétique numéro 1024. Pour la\nrécupérer, il suffit que vous utilisiez le système de stockage pérenne, auquel\non peut accéder à l\'adresse :\n\n        /bin/long-term-storage\n\nIl vous suffira de vous laisser guider.\n\nPour ma part, je n\'y comprends rien, comme je ne comprends rien à la \ncryptographie en général, mais d\'après un tuto que j\'ai trouvé sur un forum, on \nm\'a dit qu\'il fallait utiliser la méthode (standard) nommée "aes-128-cbc". On \nme demande d\'utiliser le mot de passe ``@&amp;3MiZisc&amp;\'\' (note : il est \ndisponible dans une autre pièce-jointe d\'identifiant \'password\').\n \nRenvoyez-moi vite la version chiffrée à l\'adresse ``xcasey\'\', s\'il vous \nplait.\n---\nB

In [48]:
c.get('/bin/long-term-storage/1024')

{'url': '/bin/long-term-storage/device/859ee5c7',
 'status': 'READY',
 'message': 'Storage Device 1024 is available',
 'operator': 'amanda33',
 'signature': 'MEUCIHmOuI4h5NjB/UoJidcsjWSmXp+TFQr7jYtRU4QQscN/AiEAhTRy0OPHUdI+u4AWOnNzRk5nClBcZZN+/UIGeywujlE='}

In [55]:
c.get('/bin/crypto_helpdesk/ticket/1112/attachment/password')

'@&3MiZisc&'

In [57]:
file = c.get('/bin/long-term-storage/device/859ee5c7')
file_crypter = encrypt(file, "@&3MiZisc&")
c.post('/bin/sendmail', to='xcasey', subject="file", content=file_crypter)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [58]:
c.post('/bin/crypto_helpdesk/ticket/1112/close')

'STATUS : OK --- ticket 1112 closed\n\n[[1612643620##desk-trackerd[37] UGLIX]]: SK.101:3030:1@464227|218d5bf2e44a7898c4a07157e972e2aa\n[ You have new mail ]'

In [363]:
c.get('/bin/crypto_helpdesk/ticket/1113')

'Bonjour,\n\nD\'habitude je n\'ai pas de problème avec la cryptographie de base, mais là\nj\'ai reçu un message que je n\'arrive pas à déchiffrer. J\'ai contacté\nmon correspondant, et il m\'a dit que le problème c\'est que (pour une raison\nobscure), il tient à tout prix à utiliser la méthode ``rc2-ofb\'\'. \n\nOr, je ne sais pas comment on fait pour déchiffrer ces trucs-là.\n\nLe mot de passe utilisé pour générer la clef de chiffrement est : ``#82Xkg@r_o\'\'.\n\n\nVous trouverez en pièce-jointe : \n  + "ciphertext" = message chiffré\n  + "cipher"     = procédé de chiffrement\n  + "password"   = mot de passe\n\n\nRenvoyez-moi le texte clair à l\'adresse ``moodytravis\'\'.\n\nMerci d\'avance,\n---\nKimberly Jordan\n(Cyber-)Law Enforcement Agency, service "terrorisme"\n[ You have new mail ]'

In [364]:
c.get('/bin/crypto_helpdesk/ticket/1113/attachment/cipher')

'rc2-ofb'

In [66]:
c.get('/bin/crypto_helpdesk/ticket/1113/attachment/password')

'#82Xkg@r_o'

In [68]:
text = c.get('/bin/crypto_helpdesk/ticket/1113/attachment/ciphertext')
with open ('Ticket_1113.bin', 'w') as f:
    f.write(text)
resultat = decrypt('Ticket_1113.bin', '#82Xkg@r_o', 'rc2-ofb')
print(resultat)

Cher collègue, 

Voici la liste des comptes ayant exhibé une activité suspecte cette semaine :
  + kimberlyross                     (nom complet = John Phillips)
  + kristinaparker                   (nom complet = Joseph Davis)
  + dianajackson                     (nom complet = Brenda Lopez)
  + smoore                           (nom complet = David Patterson)
  + michaeljohns                     (nom complet = Jeffrey Castillo)
  + anonymous.coward                 (nom complet = ?????? ???????)
  + joshua10                         (nom complet = Nathan Sandoval)
  + palmerjohn                       (nom complet = Bailey Brown)
  + julielee                         (nom complet = Christine Yu)

Je crois savoir que votre service dans la division "cyber-terrorisme" mène 
actuellement une enquête sur un soit-disant "leader d'une cellule terroriste 
préparant activement un attentat majeur dans de bref délais". Si mes 
informations sont bonnes, c'est dans l'espoir de l'identifier que vous av

In [69]:
c.post('/bin/sendmail', to='moodytravis', subject="Message", content=resultat)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [70]:
c.post('/bin/crypto_helpdesk/ticket/1113/close')

'STATUS : OK --- ticket 1113 closed\n\n[[1612645729##desk-trackerd[37] UGLIX]]: SK.UNUSUAL:3030:1@819491|a031fb6e068ca77447c81af46715ccb7\n[ You have new mail ]'

In [411]:
re_chalen = c.get('/bin/login/CHAP')
re_chalen

{'challenge': 'ea93e48ab3154b51810cae6cbff1e97d'}

In [412]:
plan_text = 'palmerjohn-'+re_chalen['challenge']
resultat = encrypt(plan_text, 'j^5vL+Fpv@')
print(resultat)

U2FsdGVkX1+ZnNXtrr0DzO2pU7YvyaN9KstKEaRCwalO4QIFK4BKr7H23vqxzGsz
pZgtDR9tAZlsIR5sPHH+dQ==



In [413]:
c.post('/bin/login/CHAP', user= 'palmerjohn', response= resultat)

'CHAP login successful. Access to /home/palmerjohn granted.\n[ You have new mail ]'

In [31]:
print(c.get('/bin/crypto_helpdesk'))

[initializing ticket system]
[connecting to tickets daemon]
[fetching tickets]---> 10 tickets in queue
[building UI]
[startup complete]

Hello palmerjohn,

Welcome to Obrien, Collins and Love Helpdesk User Interface.

Your pending ticket list:
-----------+-------+----------------------------------+------------------------------------
|  id      | level | client                           | mission
-----------+-------+----------------------------------+------------------------------------

If you need assistance, /doc/helpdesk is a good place to start.

[ You have new mail ]


In [7]:

def encrypt_public(plaintext, file_key):
    
    # prépare les arguments à envoyer à openssl
    # openssl pkeyutl -encrypt -pubin -inkey <fichier contenant la clef publique>
    args = ['openssl', 'pkeyutl', '-encrypt', '-pubin', '-inkey', file_key]
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=plaintext, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return base64.b64encode(result.stdout).decode()

In [36]:
c.get('/bin/crypto_helpdesk/ticket/1123')

ServerError: ERREUR 400, Une fois qu'un tunnel chiffré est ouvert, les requêtes doivent être adressées à la passerelle (/uglix/bin/gateway).
Vous avez envoyé une requête à /uglix/bin/crypto_helpdesk/ticket/1123

In [246]:
message = c.get('/bin/crypto_helpdesk/ticket/1123/attachment/message')
message

"Communiqué de l'UE du 1995-06-06"

In [34]:
key = c.get('/bin/crypto_helpdesk/ticket/1123/attachment/public-key')
with open ('key_du_relou.bin', 'w') as f:
    f.write(key)
print(key)

ServerError: ERREUR 400, Une fois qu'un tunnel chiffré est ouvert, les requêtes doivent être adressées à la passerelle (/uglix/bin/gateway).
Vous avez envoyé une requête à /uglix/bin/crypto_helpdesk/ticket/1123/attachment/public-key

In [260]:
resultat = encrypt_public(message, 'key_du_relou.bin')
print(resultat)

kturI9eQP7J5OhYM97T3PqRqTFpUwnwnK4POg6Xpsk7dhGg77rRfLzX0d5LU/7TdCtJoXjdHxPFH445fvU8GWYCV51yFByq2J/J/FwcFBubm80MPufXe+zZZkQcZamJRKrWOGZjK1JNWeZbwSyNFdrmQFD5wwcZeOcMwhiUJxGV0tLAcRnP7D6Pk6fBl2FK5HRvXdfbt5SdytODWdkt6WebFJcML0zmNiTF85CnPLeU0kpgH80fU3bEzN4jChxVPs+FhLUHH4rAZ3vSpKz7l1dHXuyuBLLzqxNvLMHIY21bQeGSmEB4oXsMtQPX2XPFhbRltYWaI/pDqJ8oo0/c0rA==


In [261]:
c.post('/bin/sendmail', to='christina04', subject="Resultat", content=resultat)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [262]:
c.post('/bin/crypto_helpdesk/ticket/1123/close')

'STATUS : OK --- ticket 1123 closed\n\n[[1612652545##desk-trackerd[37] UGLIX]]: PK.101.ENC:3030:2@156455|7e5320ed6f2a79070c286c4ace1ca79f\n[ You have new mail ]'

In [8]:
def decrypt_public(file_name, file_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'pkeyutl', '-decrypt', '-inkey', file_key]

    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(file_name, str):
        file_name = base64.b64decode(file_name)
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=file_name, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [21]:
print(c.get('/bin/crypto_helpdesk/ticket/1124'))

Bonjour,

Je suis journaliste et une de mes sources cherche à me transmettre une 
information sensible (des données qui auraient "fuité" au plus haut niveau). 
Cette personne est hautement paranoïaque, et n'accepte que de communiquer via
des emails chiffrés. Mon problème est que  je n'ai jamais réussi à générer une
paire de clefs asymétriques, et je ne peux pas pas rencontrer ma source en 
personne pour échanger une clef secrète.

Vous devez donc entrer en contact avec ma source, ``cknapp''.
Il suffit de lui envoyer un email (chiffré) contenant la chaine "informations 
très très très sensibles". Elle comprendra de quoi il s'agit, et vous enverra en retour un 
email chiffré. Vous devez déchiffrer cet email et me le transmettre à l'adresse
``anthonyjessica''.

Vous trouverez sa clef publique avec le programme /bin/key-management.

Son adresse est en pièce jointe (id="contact").
---
Chloe Mcbride, 
rédacteur de la rubrique "Horizontal responsive projection" chez Nguyen, Cruz and Osborn
[ 

In [12]:
def gen_key_private():
    args = ['openssl', 'genpkey', '-algorithm', 'RSA', '-pkeyopt', 'rsa_keygen_bits:2048']
    result = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    with open('key_private.pub', 'w') as f:
        f.write(result.stdout.decode())
        
def trouver_key_public(file_key_private): 
    args = ['openssl', 'pkey', '-in', file_key_private, '-pubout']
    result = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode()
    

In [23]:
addresse = c.get('/bin/crypto_helpdesk/ticket/1124/attachment/contact')
addresse

'cknapp'

In [599]:
pk_contact = c.get('/bin/key-management/{}/pk'.format('cknapp'))
with open("key_source.bin", 'w') as f: 
    f.write(pk_contact)

In [600]:
gen_key_private()
key_public = trouver_key_public('key_private.pub')
#print(key_public)
c.post('/bin/key-management/upload-pk', public_key = key_public, confirm = True)

'OK'

In [601]:
message = 'informations très très très sensibles'
message_crypte = encrypt_public(message, "key_source.bin")

In [602]:
c.post('/bin/sendmail', to='cknapp', subject="Message", content=message_crypte)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [605]:
reponse = c.get('/home/palmerjohn/INBOX/11439/body')
print(reponse)

HCI1OiZlAJjxG/6H6fZUbkJJI3KOdSxxNQw/7fRrjGG5cK+sywPnZzdnyYs4ooPSjjTjCRmEDACE9SGn54DmuFcE766phufqFKG6hgBf1coHrIsLefYqCgHNnhmGWFWYQaipmIxEgGV/O/W2eNZHcop5ERMKcI8hmOPXVFd+GmPqUP+mUrWenC43APp96xb/jIlAH3AoZlg3vgmiMe2G8jcXpcibou7aWQ5/95LnaEhaVpn5hjw9EzSrSKNRjmRWtoOa03Xyg0z45UN0maGrgw19wSOeyqwDpSMTYcRVUxmd3LDgtA5OfJrdRN2E02L9Z92fSezuGP1r7EG2U+p1/Q==


In [608]:
reponse_decrypte = decrypt_public(reponse, 'key_private.pub')
print(reponse_decrypte)

Communiqué de l'UE du 1985-06-06


In [611]:
print(c.post('/bin/sendmail', to='anthonyjessica', subject="Reponse", content=reponse_decrypte))

STATUS: OK --- mail delivered succesfully
[ You have new mail ]


In [610]:
c.post('/bin/crypto_helpdesk/ticket/1124/close')

'STATUS : OK --- ticket 1124 closed\n\n[[1612709376##desk-trackerd[37] UGLIX]]: PK.101.DEC:3030:3@816811|17ebad25f1d4b2c0f7e8ad2f8ef325f3\n[ You have new mail ]'

In [621]:
print(c.get('/bin/crypto_helpdesk/ticket/1125'))

Bonjour,

Je suis le rédacteur en chef de la rubrique "Horizontal responsive projection" chez Nguyen, Cruz and Osborn.
Il se peut que vous ayez déjà eu affaire avec un de mes subordonnés. Nous avons 
passé un accord avec une source disposant d'informations sensibles. En échange 
des ses informations, nous devions lui communiquer un document l'intéressant 
particulièrement. ``Quid pro quo'', en somme. Nous ne savons pas très bien ce 
que cette personne à l'intention de faire de ce document, mais ce n'est de toute 
façon pas notre problème.

Cependant, il s'agit d'informations sensibles, et il est donc exclu de les 
envoyer en clair. Heureusement notre source possède une paire de clefs asymé-
triques. Il faut donc transmettre le document en utilisant un chiffrement hybride
(en effet, il est trop gros pour pouvoir être chiffré tel quel avec une clef 
publique). cf. /doc/hybrid.

Son identité est en pièce-jointe (id="contact"), de même que le fichier qu'il 
faut lui transmettre en pièce-jo

In [622]:
contact = c.get('/bin/crypto_helpdesk/ticket/1125/attachment/contact')
file = c.get('/bin/crypto_helpdesk/ticket/1125/attachment/reciprocity')
print(contact)

brandon09


In [623]:
key_destinateur  = c.get('/bin/key-management/brandon09/pk')
key_secrete_jetable = 'MIIBIjANBgkqhkiG9w0IIBCgKCx2EqhHR4GdIqiKgxKiuqG'
with open ('key_public.bin', 'w') as f:
    f.write(key_destinateur) 

session_key = encrypt_public(key_secrete_jetable, 'key_public.bin')
open('session_key_file.bin', 'wb').write(session_key.encode())

344

In [637]:
message_chiffre = encrypt(file, key_secrete_jetable)
dictionnaire = {"session_key":session_key, "payload": message_chiffre}
c.post('/bin/sendmail', to='brandon09', subject="Fichier", content=dictionnaire)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [640]:
print(c.get('/home/palmerjohn/INBOX/11466'))

----------------------------------------------------------------------
Message #11466.


Date: 07-02-21893 15:46:04
From: brandon09
Subject: Re: Fichier

Thanks, pal.




In [641]:
c.post('/bin/crypto_helpdesk/ticket/1125/close')

'STATUS : OK --- ticket 1125 closed\n\n[[1612712783##desk-trackerd[37] UGLIX]]: HYBRID:3030:3@126793|b4607b2fbfbde546172b59031f2ccc12\n[ You have new mail ]'

In [865]:
print(c.get('/home/palmerjohn/INBOX/11468'))
mail = c.get('/home/palmerjohn/INBOX/11468/body')

----------------------------------------------------------------------
Message #11468.


Date: 07-02-21893 15:46:23
From: janice66
Subject: Votre nouveau contrat de travail

-------------------- Contrat de travail --------------------

Entre les soussignés :

*) CLEA, une agence officielle de UGLIX, représentée par [[---- REDACTED ----]],
   agissant en sa qualité de "UNKNOWN", ci-après désigné "L'Agence"

*) palmerjohn, ci-après désigné "Le consultant"


Il est arrêté et convenu ce qui suit :

Article 1 : Engagement
----------------------
Vous êtes engagé(e) au sein de l'agence dès la signature du présent contrat.


Article 2 : Fonctions exercées
------------------------------ 
Vous exercerez les fonctions de "Consultant" (non-admin).


Article 3 : Nature de la mission
---------------------------------
L'agence confie au consultant une mission consistant à traiter des 
[[---- REDACTED ----]], apporter son expertise aux [[---- REDACTED ----]].
Éventuellement, procédeder à des analyses 

In [13]:
def signature(document, secret_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'dgst', '-sha256', '-sign', secret_key]
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(document, str):
        document = document.encode('utf-8')
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=document, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return base64.b64encode(result.stdout).decode()

In [19]:
gen_key_private()
key_public = trouver_key_public('key_private.pub')
c.post('/bin/key-management/upload-pk', public_key = key_public, confirm = True)

'OK'

In [20]:
signature_numerique = signature(mail, 'key_private.pub')
print(signature_numerique)
c.post('/bin/sendmail', to='janice66', subject="Signature", content=signature_numerique)

TypeError: memoryview: a bytes-like object is required, not 'function'

In [64]:
print(c.get('/home/palmerjohn/INBOX/11519'))

----------------------------------------------------------------------
Message #11519.


Date: 07-02-21893 17:58:05
From: janice66
Subject: Re: Signature

Thanks. Your signature is archived. You are now a member of CLEA. Go get to work : /bin/police_hq. 
[[1612720685##robot-clerk[55] UGLIX]]: SAV.BYE:3030:2@782462|59d56e06d7a4acc01a18060551936061




In [19]:
print(c.get('/bin/police_hq'))

[initializing ticket system]
[connecting to tickets daemon]
[fetching tickets]
---> 1 ticket in queue
[building UI]
[startup complete]
      ,   /\   ,          Hello palmerjohn,
     / '-'  '-' \
    | UGLIX CLEA |        Agency: UGLIX (Cyber-)Law Enforcement Agency
    \    .--.    /        Rank: consultant
     |  ( 42 )  |         Admin privilege: no
     \   '--'   /
      '--.  .--'
          \/              Your ticket list:
-------+-------+-----------------------------------------------------------------------
|  id  | level |  mission
-------+-------+-----------------------------------------------------------------------
| 1496 | XXXXX | XXXXXXXXXX SECURITY TOO LOW XXXXXXXXXX USE THE STP PROTOCOL XXXXXXXXXX

If you need assistance, [---REDACTED---] is a good place to start. A debriefing is shown
when tickets are closed. You may be assigned new tickets in the future.
[ You have new mail ]


In [86]:
print(c.get('/bin/police_hq/ticket/1172'))

Nous menons actuellement une enquête sur des faits de cyber-terrorisme 
(présumé). Une des personnes auxquelles nous nous intéressons a envoyé des 
messages chiffrés par la méthode du "one-time pad". Nous considérons cette 
attitude avec suspicion : pourquoi employer une technique "incassable" et peu 
pratique, à moins d'avoir des choses très graves à dissimuler vis-à-vis des plus
hautes autorités ?

Quoi qu'il en soit, il semble que nos cibles aient commis une grosse erreur. Une
taupe infiltré dans le réseau que nous essayons de démanteler nous a révélé que
la formation à la cryptographie de nos suspects présumés est plus que
rudimentaire. Ils ont bien compris que le "one-time pad" est une méthode fiable,
ils ont également bien compris qu'il faut une source d'aléa de bonne qualité
pour générer le masque, mais ils n'ont pas compris qu'il faut changer de masque
à chaque fois !

Lors de l'enquête nous avons saisi des documents chiffrés avec le MÊME masque.
Notre équipe de h4x0rz à réussi

In [132]:
print(c.get('/bin/police_hq/ticket/1172/attachment/client'))

ServerError: ERREUR 403, Session not opened.

In [88]:
document_A = c.get('/bin/police_hq/ticket/1172/attachment/exhibit-A')
document_B = c.get('/bin/police_hq/ticket/1172/attachment/exhibit-B')

In [96]:
document_A_bytes = base64.b64decode(document_A)
document_B_bytes = base64.b64decode(document_B)
print(type(document_A_bytes))

<class 'bytes'>


In [100]:
M1_M2 = [chr(a^b) for a,b in zip(document_A_bytes, document_B_bytes)]
M1_M2 = ''.join(M1_M2)
M1_M2 = M1_M2.encode()

In [117]:
b0 = b""
for i in range(0, len(M1_M2)):
    b0 += b"0"
    
b1 = b""
for i in range(0, len(M1_M2)): 
    b1 += b'1'
    
M1_M2_0 = [chr(a^b) for a,b in zip(M1_M2, b0)]
M1_M2_0 = ''.join(M1_M2_0)
M1_M2_0 = M1_M2_0.encode()

M1_M2_1 = [chr(a^b) for a,b in zip(M1_M2, b1)]
M1_M2_1 = ''.join(M1_M2_1)
M1_M2_1 = M1_M2_1.encode()

print("1er partie:")
print((M1_M2_0).decode())
print("2e partie:")
print((M1_M2_1).decode())


1er partie:
-oIpwd$sU!knts!dy`muóò²óòóò³óò¨!m&`óò²óòóò³óò©ed!qsnr`óò²óòóò³óò®rdQnts!eóò²óòóò³óò¨qm`hsd!`t!qsng`od!`tssi chdo!pt&`ty!hehnurM` dóò²óòóò³óò©couvdsud!`mnsr!wnhmóò²óòóò³óò¡!qui!usaum`uhrdDu!gdsldoudot ed!lóò²óòóò³óò«me!du!mdr!bthsr!du ldr!qd`tx
Et!wnhrho!md!Q`qou!rtóò²óòóò³óò¦d!l'`qnqixrdMd!wtmfaisd!r&dotóò²óòóò³óò«ud!óò²óòóò³óò¡!wntmnhs!ees!wdsr!cdatyHm!fsdmnuu`hu ld!q`twre aux bords dd!m`!TalhrdElle effraie le Cdssx!bnlle les MorvandiauxM`!Fsóò²óòóò³óò©bd de Platon óò²óòóò³óò  couq!róò²óòóò³óòºs!o&dst poinu!rotte
Gr`uuds!md!qarchemio!edviendr`!r`!l`snute
Le cnmnoel s'óò²óòóò³óò¨qnofd!to!blason e`os la mahoNo!sdfsette óò²óòóò³óò  m` fin ler!`fsdrudr bicoqudrExaltenu!m&d
2e partie:
,nHqve%rT jour exaltòó³òóòó²òó© l'aòó³òóòó²òó¨de prosaòó³òóòó²òó¯se
Pour dòó³òóòó²òó©plaire au profane aurrh!bien qu'aux idiots
La!eòó³òóòó²òó¨bntwerte alors voilòó³òóòó²òó  pth tr`tla

In [31]:
i = c.get('/bin/police_hq/ticket/1172/attachment/i')
j = c.get('/bin/police_hq/ticket/1172/attachment/j')
print('i = {} et j ={}'.format(i, j))

i = 10 et j =4


In [32]:
image = c.get('/bin/police_hq/ticket/1172/attachment/sample')
print(image)
indication_0 = c.get('/bin/police_hq/ticket/1172/attachment/indication-0')
print("indication 0 = {}".format(indication_0))
indication_1 = c.get('/bin/police_hq/ticket/1172/attachment/indication-1')
print("indication 1 = {}".format(indication_1))
indication_2 = c.get('/bin/police_hq/ticket/1172/attachment/indication-2')
print("indication 2 = {}".format(indication_2))



P1
24 7
000000000000000000000000011110011110011110011110010000010000010000010010011100011100011100011110010000010000010000010000010000011110011110010000000000000000000000000000

indication 0 = Respirez calmement.
indication 1 = Combinez les deux messages pour éliminer le masque.
indication 2 = Vous ne connaissez pas précisément le contenu de l'image, mais vous
savez que l'essentiel est constitué d'une longue séquence qui ne peut
contenir que deux caractères différents.


In [109]:
contact = c.get('/bin/police_hq/ticket/1172/attachment/client')
print(contact)

{'name': 'Jeffrey West', 'company': 'Graham Group', 'email': 'anthonywebb'}


In [141]:
c.post('/bin/sendmail', to='anthonywebb', subject="mots", content="blason")

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [143]:
mail("palmerjohn", "12667")

----------------------------------------------------------------------
Message #12667.


Date: 12-02-21893 16:11:55
From: anthonywebb
Subject: Re: mots

Super, merci ! Vous pouvez fermer le ticket.




In [144]:
c.post('/bin/police_hq/ticket/1172/close')

'STATUS : OK --- ticket 1172 closed\n\nDEBRIEFING\n----------\n\nIl semble que nous ayons été conduit sur une fausse piste. Le contenu chiffré\nne semble pas critique.\n\nCONCLUSION: Affaire classée\nISSUE : sans suite\n\n[[1613148073##desk-trackerd[37] UGLIX]]: 2X.PAD:3030:3@335022|e20e081e53148beb560d56c204d700ef\n[ You have new mail ]'

In [177]:
print(c.get('/bin/police_hq/ticket/1173'))

Nous enquêtons actuellement sur une série de délits bancaires. Il semble que des
fraudeurs aient trouvé un moyen de désactiver toute une série de vérifications
de sécurité dans ces terminaux de paiement, ce qui leur permet de payer avec
de fausses cartes de crédit. Ces cartes indiquent un numéro de compte bidon, et
du coup les banques doivent assumer des transactions et elles ne peuvent débiter
personne.

Les banques qui coopèrent avec nos services ont compilé une base de données de
transactions potentiellement frauduleuses. Nous souhatons que vous en examiniez
un extrait et que vous déterminiez si chaque transaction est légitime.

En vous connectant à cette adresse :

    /bin/banks/forensics

Vous obtiendrez un lot de transactions à traiter. Plus précisément, vous
obtiendrez :
    + un identifiant de lot (identifier)
    + une liste de numéros de cartes (card-numbers)

Une transaction est un triplet formé :
	+ du numéro de la carte
	+ d'un "challenge" envoyé par le terminal de paieme

In [179]:
certificat_autorite = c.get('/bin/banks/CA')
#print(certificat_autorite)

In [14]:
def verifie_certificat(certificats, certificatsA, certificatsB):    
    reponse = True 
    if isinstance(certificats, str):
        certificats = certificats.encode('utf-8')
    #openssl verify -trusted <fichier_1> -untrusted <fichier_2>
    args = ["openssl", "verify",  "-trusted", certificatsA, "-untrusted", certificatsB]

    result = subprocess.run(args, input=certificats, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        reponse = False 
    if "error" in result.stderr.decode(): 
        reponse = False
    return reponse 

In [15]:
def recuperer_sujet_certificat(certificats): 
    args = ["openssl", "x509",  "-subject", "-noout"]
   
    if isinstance(certificats, str):
        certificats = certificats.encode('utf-8')
    
    result = subprocess.run(args, input=certificats, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [18]:
def recuperer_cle_public(certificats): 
    args = ["openssl", "x509",  "-pubkey", "-noout"]
    if isinstance(certificats, str):
        certificats = certificats.encode('utf-8')
    
    result = subprocess.run(args, input=certificats, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)


    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [19]:
def recuperer_text_certificat(certificats): 
    args = ["openssl", "x509",  "-text", "-noout"]
    if isinstance(certificats, str):
        certificats = certificats.encode('utf-8')
    
    result = subprocess.run(args, input=certificats, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [47]:
def verification_signature_carte(key_certificat, signature, challenge): 
    #openssl dgst -sha256 -verify public_key.pem -signature signature.bin
    with open("key_certificat.bin", "w") as f: 
        f.write(key_certificat)
    args = ["openssl", "dgst", '-sha256',  "-verify", "key_certificat.bin","-signature", signature]
    if isinstance(challenge, str):
        challenge = challenge.encode('utf-8')
    
    result = subprocess.run(args, input=challenge, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    resultat = result.stdout.decode()
    if 'Failure' in resultat: 
        resultat = False
    else : 
        resultat = True 
    return resultat

In [21]:
def verification(): 
    donnee  = c.get('/bin/banks/forensics')
    identifier = donnee["identifier"]
    liste = donnee['card-numbers']
    CA = c.get("/bin/banks/CA")
    with open("certificatsA.txt", "w") as f: 
            f.write(CA)
    CA = "certificatsA.txt"
    reponse = []
    for i in liste: 
        #données publiques 
        info_publique = c.get('/bin/banks/card-data/'+i)

        #info : 
        nom_de_banque = info_publique["bank-name"]
        certificat_banque = info_publique["bank-certificate"]
        certificat_carte = info_publique["card-certificate"]
        challenge = info_publique["challenge"]
        signature = info_publique["signature"]
        num_carte = info_publique["card-number"]
        
        with open("certificats_carte.txt", "w") as f: 
            f.write(certificat_carte)

        with open("certificats_banque.txt", "w") as f: 
            f.write(certificat_banque)

        
        signature = base64.b64decode(signature)
        with open("signature.bin", "wb") as f: 
            f.write(signature)
            
        key_certificat = recuperer_cle_public (certificat_carte)
        with open("public_key.pem", "w") as f: 
            f.write(key_certificat)
            
        with open("challenge.txt", "w") as f: 
            f.write(challenge)

        certificats_carte_f = "certificats_carte.txt"
        certificat_banque_f = "certificats_banque.txt"
        
        #Verifications : 
        v_certificat_carte = verifie_certificat(certificat_carte, CA, certificat_banque_f)
        v_nom_in_certificat = nom_de_banque in recuperer_sujet_certificat(certificat_banque)        
        v_signature_carte = verification_signature_carte("public_key.pem", "signature.bin",challenge)
        v_num_carte = num_carte in recuperer_sujet_certificat(certificat_carte)
        reponse.append(v_certificat_carte&v_nom_in_certificat&v_signature_carte&v_num_carte)
    return reponse, identifier
    

In [503]:
statuses, identifier  = verification()
print(statuses)

[False, False, True, True, False, True, True, True, True, False, False, False, False, False, False, True, True, True, False, True, False, False, False, True, False, False, True, False, False, False, False, False, False, True, False, True, True, False, False, True, True, False, True, True, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, False, False, False, True, True, False, True, True, False, True, False, True, True, True, False, True, False, False, True, True, False, False, True, False, True, False, False, True, False, False, True, True, False, True, True, True, False, True, True, True, False, True, False, False, True, True, False, False, True, False, True, True, True, False, False, True, False, True, False]


In [422]:
#TRUE
liste_de_carte = ['5413-4092-2581-7982'] 
statuses = verification(liste_de_carte)
#print(statuses)

b'0E\x02!\x00\xdef\x11 *\xed\xf5A^\xd6Q\xaf<\xa6\n\xda\xa4]\x8d\xa835\x90\xd9R\x01"q\xc8s\xd6\xc9\x02 `\x8dO\xe3i\xc0\xa3\x19\xb2u\x8a\x83\xc7\xf3\xbd$m\xba\xa3\xd7OV\x07\x9a\x8c8\x85ptEt\xd7'
<class 'str'>
v_certificat_carte : True  
 v_nom_in_certificat: True 
  v_signature_carte :True 
 v_num_carte :True


In [423]:
#FALSE 
liste_de_carte = ['4190-9702-0375-2474'] 
statuses = verification(liste_de_carte)
#print(statuses)

b'0F\x02!\x00\xa5\x19\xe7j\x8be\x17+Yb\x8bq\x8d\x87\x14\xcb\x95L\xedXx\xca1\xd2\xf2]\x01)K\x99\x7f\x1c\x02!\x00\x89\x97@n~G\xd8/\xde4\xd8\xf5\x91\xebO\x84\x88\xfb\xbfX\xf9\xf7`j\xd2<\xec\xdb;{\x1d3'
<class 'str'>
O = "Dawson, George and Clark", OU = Cards Management, CN = 4190-9702-0375-2474
error 7 at 0 depth lookup: certificate signature failure
140693836895424:error:0D0C5006:asn1 encoding routines:ASN1_item_verify:EVP lib:crypto/asn1/a_verify.c:170:

v_certificat_carte : False  
 v_nom_in_certificat: True 
  v_signature_carte :True 
 v_num_carte :True


In [344]:
dico = c.get("/bin/banks/forensics/sample")
print(dico)

{'transaction': {'card-certificate': '-----BEGIN CERTIFICATE-----\nMIIBoTCCAUgCFHPdIli7qNsWaLI7V66mTojQX5p/MAoGCCqGSM49BAMCME4xITAf\nBgNVBAoMGERhd3NvbiwgR2VvcmdlIGFuZCBDbGFyazEXMBUGA1UECwwOS2V5IE1h\nbmFnZW1lbnQxEDAOBgNVBAMMB0JhbmsgQ0EwHhcNMjEwMjEyMjA0OTAwWhcNMjIw\nMjEyMjA0OTAwWjBcMSEwHwYDVQQKDBhEYXdzb24sIEdlb3JnZSBhbmQgQ2xhcmsx\nGTAXBgNVBAsMEENhcmRzIE1hbmFnZW1lbnQxHDAaBgNVBAMMEzQxOTAtOTcwMi0w\nMzc1LTI0NzQwVjAQBgcqhkjOPQIBBgUrgQQACgNCAAQh+2rbRz6p4xMBauUd0M3N\na6oD8A/+cSj7g2A8AM/7eD6oUt+L77oucqXRW0OWSJm8lpTWxfqLsEJqcyJa+z4b\nMAoGCCqGSM49BAMCA0cAMEQCIFeydRUIaEjyOICuG2+ZcXqAxfrkf7Vsjcm9lLys\n8z3OAiBsEoDXebKkSqnLlj3tUeXHnyPu4SAJnJygSJMqFaVgFA==\n-----END CERTIFICATE-----\n', 'bank-name': 'Dawson, George and Clark', 'card-number': '4190-9702-0375-2474', 'bank-certificate': '-----BEGIN CERTIFICATE-----\nMIICATCCAaegAwIBAgIUNQ9yTqcMiuR+bbOZyxgAHcFjtzkwCgYIKoZIzj0EAwIw\nTjEkMCIGA1UECgwbSGlnaGVyLW9yZGVyIEFic3RyYWN0IExvZ2ljMQ4wDAYDVQQL\nDAVVR0xJWDEWMBQGA1UEAwwNVUdMSVggcm9vdCBDQTAeFw0yMTAyMTIyMDQ5

In [504]:
username = c.get('/bin/police_hq/ticket/1173/attachment/client')
print(username)

{'name': 'Michael Hernandez', 'company': 'Sandoval-Orozco', 'email': 'stevensdavid'}


In [505]:
c.post('/bin/sendmail', to='stevensdavid', subject="Test", content={"identifier" : identifier, "statuses":statuses})

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [508]:
mail("palmerjohn", "13223")

----------------------------------------------------------------------
Message #13223.


Date: 12-02-21893 22:07:18
From: stevensdavid
Subject: Re: Test

Super, merci ! Vous pouvez fermer le ticket.




In [513]:
c.post('/bin/police_hq/ticket/1173/close')

'STATUS : OK --- ticket 1173 closed\n\n[[1613168213##desk-trackerd[37] UGLIX]]: CREDIT.FRAUD:3030:4@553791|74431866ef34f28da649cb305fac3bc4\n[ You have new mail ]'

In [26]:
print(c.get('/bin/police_hq/ticket/1174'))

Ceci n'est pas lié à aucune enquête en cours. Il s'agit plutôt d'une opération
de maintenance.

Ted, un de nos techniciens doit se rendre dans un data-center où se trouvent des
noeuds UGLIX pour intervenir sur l'un d'entre eux. Seulement, pour cela, il doit
agir en tandem avec un assistant, qui accède à distance à la la console de
pilotage du matériel. Ted est parfois un peu caractériel, mais tout devrait bien
se passer. Il est très apprécié de la plupart des agents.

On ne peut accèder à la console que si on a ouvert une session avec un protocole
chiffré, comme par exemple le protocole STP (cf. /doc/STP).

La console est accessible à l'adresse /service/hardware/status.

Cette mission sera accomplie lorsque l'intervention du technicien sera terminée.
N'oubliez pas de passer au debriefing.

La communication avec le technicien n'est pas forcément très bonne. N'hésitez pas
à rafraichir votre vue de temps en temps.
---
Gloria Turner, CLEA agent.
"Don't break the (cyber-)law"

PS : un conse

In [27]:
nonce = c.post("/bin/login/stp", username= "palmerjohn")

In [28]:
K = "j^5vL+Fpv@-"+nonce

In [38]:
def encrypt2(plaintext, passphrase, cipher='aes-128-cbc'):

    # prépare les arguments à envoyer à openssl
    pass_arg = 'pass:{0}'.format(passphrase)
    args = ['openssl', 'enc', '-' + cipher, '-pass', pass_arg, '-pbkdf2']
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=plaintext, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout

In [30]:
requete = {'method': 'GET', 'url': '/bin/login/stp/handshake'}
requete_json = json.dumps(requete)
print(requete_json)
requete_chiffre = encrypt2(requete_json, K)
print(type(requete_chiffre))

{"method": "GET", "url": "/bin/login/stp/handshake"}
<class 'bytes'>


In [31]:
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
print(resultat)

b'Salted__`\xf3\xdc~\xb0l\x7f8\xa9]A\xf7S\x14n\xc4\x9a6\x82\xa3-\xc6\x8f\x92V\xbaM\x9f\xe7\x17\x95&\xc5\xb1\xb1\xe0\r\xe1\xbe8gb\x14T\x9a\xed.\x18\xb3&D\x03\xbbM\x83E\x99X\xc5\x18\xcf\x91^\xe2\x1f\xe4\xc9<GhJ\xaa\xd4\x8b\xe3\non\xdf\xd3\xa9\x866\xf9\x13\x0bA\x17'


In [34]:
#resultat = base64.b64encode(resultat).decode()
#print(resultat)
#with open("reponse.txt", 'w') as f: 
#    for i in range(0, len(resultat)): 
#            if (i % 64 == 0): 
#                f.write("\n")
#                f.write(resultat[i])
#            else : 
#                f.write(resultat[i])
reponse = decrypt("reponse.txt", K)

In [35]:
print(reponse)

STP login successful. Access to /home/palmerjohn granted.
[ You have new mail ]


In [187]:
requete = {'method': 'GET', 'url': '/service/hardware/status'}
requete_json = json.dumps(requete)
print(requete_json)
requete_chiffre = encrypt2(requete_json, K)
print(type(requete_chiffre))

{"method": "GET", "url": "/service/hardware/status"}
<class 'bytes'>


In [188]:
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
print(resultat)

b'Salted__M\xc1\xc6\x08\xbe\xce\x86pYs\xd2n\xb9\x8e\x8b\xc6\xc0\x18"\xf1\xdd\x7f[^\x08\xe3L\xdb\x1f,+:K\x08\xea-{\x9aPa\xfay\xfey6\xe5\xf33\xc8\xb9/k\xaf\xb8\x8c\x99Z\xf28\x8e\xf2:#O~\xcf\xf1\xbd\x98\xab\x93/\x8ec_kQ\xae\xa1^Ol<9\\\xa8\x00\x8bQ\xd5k\xe8\xa2\xf5,R+\xe1\x1d\xbb\xea6\rCIY\xa1\xb9K\x14\x04;\xe7\x0e\xba|D\xdf(b\xdce>\x98\xfav\x7f\xf7\x10\xf6\xc8\x97h\x19\x02\xf2\xe6q\x11h\xacz\tj\xa0\x99\xc0\x19\xb8\x83\x06\xa4I\xff\xe9\xb1\xf6\x8d\xd9F1\xf7j`\x0c\x95X6\xd3\xdbmT,6\xc2\xd7\x00\xe7e&\xfcf\x88\xadxr"@\xae\xd9\xe8Y!\xa5o\xe1\x90\xfb(\xf4ET\xb6\xd7k\xea\x10(\xeb\x9a\\v\xf2+\xe5"=\xef\x1a\x91\xb7\xe3\x85S\xd0\x16up\xd0\xc0\\\x89\x91k\xae\xc6\xad\x91d\x04\x83Q\x1d\x05=[4\xec\x16m\xf3\xf5\xffA\xa9\x8aHtW]\x05\xa8\xdb\x135\x92"\x0e\x99?\xa9?>\x9c\xa1>Z\xc9)\x9d\x95\x8f\x10O\xfepBCS\x13\xc7\x867\xc0\xfb]\xb3\x94\xb3\x03\x88f\xf21\xfd\xd0\x89\xcb\xa2\xa7\xeb\x8d\x07\xa3\xef\xd9\x93\x8a>\xfa\xc0\xf2lE\x16\xca\x98\x01\x13\nE\x97\x8bF\xc6y\xa0f\xcc\x82\xd4%\xee\xff\xe1\x12\x8e\x1b}M\xd8

In [189]:
resultat = base64.b64encode(resultat).decode()
print(resultat)
with open("reponse.txt", 'w') as f: 
    f.write(resultat)

U2FsdGVkX19NwcYIvs6GcFlz0m65jovGwBgi8d1/W14I40zbHywrOksI6i17mlBh+nn+eTbl8zPIuS9rr7iMmVryOI7yOiNPfs/xvZirky+OY19rUa6hXk9sPDlcqACLUdVr6KL1LFIr4R276jYNQ0lZoblLFAQ75w66fETfKGLcZT6Y+nZ/9xD2yJdoGQLy5nERaKx6CWqgmcAZuIMGpEn/6bH2jdlGMfdqYAyVWDbT221ULDbC1wDnZSb8ZoiteHIiQK7Z6FkhpW/hkPso9EVUttdr6hAo65pcdvIr5SI97xqRt+OFU9AWdXDQwFyJkWuuxq2RZASDUR0FPVs07BZt8/X/QamKSHRXXQWo2xM1kiIOmT+pPz6coT5aySmdlY8QT/5wQkNTE8eGN8D7XbOUswOIZvIx/dCJy6Kn640Ho+/Zk4o++sDybEUWypgBEwpFl4tGxnmgZsyC1CXu/+ESjht9Tdhp+Jm6rCLw3M6VdKp5nYOk0ZDbbojodXt5xwl5U2QjABFgT1wkuctJQT0XkohwiriQSxPyFeRZvD1QqfFLlYDW+17rzUxul9VGdi82asTE85wQHO5VciJ+8kodH0p0uN91ZnNLCaR7gNYbEcyNmGRPHhP2nRhUxx2X8C1dJWNzYyphA5FrjSIKlrtzuzh+TOWj2VUKb20aCmQ+DQpg8i1Ksx8u5ccSTkL7wzK1TA/BeUnMtnvRo0Y03vkL6QvBHaoJfcAW4E0pME3hMqdOwrFGdubNm4MCyy0oQ6DLtpzgvtVAUEca76e7LHT/djoOCAXKMwaot6SRVdE54j6Cb/8HQrHpTQ8Jeaq+vFF/Ewv/4TlGT/AP/o2TUos4cXpk7CVAGLh4p1OFUK0YxKWKzx3eEJZh0S1RPkej7Amx4U4KhNRwmGzH2pe+rWNTFEdeiLjO1nSK8PpS/zJjGrgvINMXrvnPfVEHR2KP6FXDFCnyq2V85Dcwa46chTYSt64BWor0NdYDotty

In [190]:
reponse = decrypt("reponse.txt", K)

In [191]:
print(reponse)

┌──────────────────────────────Server Room Layout──────────────────────────────┐
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ S │ S │ S │ S │ o │ S │ S │ S │ S │0    f│ X │ ! │ X │ S │ o │ o │ X │ S │ X │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│   │ S │ S │ S │ S │ S │ S │ S │ S │1    e│ S │ X │ S │ X │ S │ X │ S │ X │ S │ 
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ . │ X │ . │ X │ X │ X │ X │ X │ X │2    d│ X │ X │ X │ X │ . │ X │ X │ X │ X │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│ S │ S │ o │ S │ S │   │ S │ S │ S │3    v│ X │ X │ X │ X │ X │ X │ X │ X │   │ 
│                                   │      │                                   │ 
│                

In [192]:
requete = {'method': 'GET', 'url': '/service/hardware/action'}
requete_json = json.dumps(requete)
print(requete_json)
requete_chiffre = encrypt2(requete_json, K)
print(type(requete_chiffre))
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
print(resultat)
resultat = base64.b64encode(resultat).decode()
print(resultat)
with open("reponse.txt", 'w') as f: 
    f.write(resultat)

{"method": "GET", "url": "/service/hardware/action"}
<class 'bytes'>
b'Salted__\xb1`\x9c_\xd2\xad;\x0f\x9co\xdf\xdf*E\xa0 \xc2N\x87}\xb4&c\xf3z\'D\x1bW\xf7,3\xbdH\xf1`36\xf5\xf3\xb7\xb0kL\xaf\xaer\x14+4\xb3=\x10Pc\xf2\xea\xe2{\xacQ\x9f\x80\x8dz\x94/\x10{\xe6\x14\xf0\xa8\x8eR\x9d;\xe9\xc3\xe9e\x12\xd5%\x9a\x88*\xca@\x97)\x07&\x1f\x0c>[\xdc\xb8H\xbcA\x03\x8fBT\x89\x068$J\xae\x01\xb7V\xb5t\x00\xa7z">\xb1\xe9\xce%\xbaY\xd7$_Q=\x82\xc2gr\xcah\xd13w\xdd\xcb\xf3\xbd\xd4U\xfc\xf8\\\xd5\x95\xd4\x89\x9d\xb8\xec\x17\xa4<\x7f\xb9=\x7f\xe3\xaa\xcbb\x7ft\xa1(nT\xccgk\xb0=\x0f\xd7X{\xbdr\xaa@D\x07\xee\xea\xd7\xea\x7f\x8eV.\xcf\xd4\x12{\xe2I(\x9b\x87\x06\xd3i\xf7\x9e^\xa8\x18d[\x8a\xe5bD\xfe\x85\\G\xfd[\x80\xbd.\xd7\xad\xca\xea\xd1\x144\x97\xa8\xedU!\xa6\xffyP\xa1\xaeHc\xd7X\xe5|\xec\xe7\x8d\xc6W\xa5vF\x1d=\xa0\xe3\xb3\xa0\xd2\xa7\x92\xf5;-\x08Az0\xb1\xdd\xf2\xf8\x14\xa4\xb7\xc6V;\r:\t0\xe0j\xb9\x8e0\xe3%\t\x93`w\x14\x87BzRV\xae\xb2\x86\x03!\x14`<\xf5JR\xebq\xc3\x97\x04w*\xdc\xa9_~\x87\xfd\x9eKIc\x94\

In [193]:
reponse = decrypt("reponse.txt", K)
print(reponse)

This is the server-room hardware control panel. 

An "action" is performed on an "item". To perform an action, send a
POST request to :

    /service/hardware/action/<item>

with an "action" argument. The possible <item> are:
    DOOR, LIGHT, VOLTAGE, SERVER-GROUP, SERVER, AIR-CONDITIONNING


The value of the "action" argument determines what the effect. The possible
values are <item>-specific.

To get more help about a specific <item>, send a GET request to the same adress.

In case of emergency (life- or equipment-threatening situation), go to:

    /service/hardware/911

Your actions are visible on the status view.


In [23]:
def commande_stp(method, url, nom_fichier):
    requete = {'method': method, 'url': url}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        f.write(resultat)

In [328]:
commande_stp("GET", "/service/hardware/action/S", "Server.txt")

In [329]:
reponse = decrypt("Server.txt", K)
print(reponse)

Possible values: lock / unlock.
Additional required argument: "n" (the number of the server group).


In [330]:
requete = {'method': 'POST', 'url': '/service/hardware/action/SERVER-GROUP', 'args': {'action': 'unlock', "n": 0xa}}
requete_json = json.dumps(requete)
print(requete_json)
requete_chiffre = encrypt2(requete_json, K)
print(type(requete_chiffre))

{"method": "POST", "url": "/service/hardware/action/SERVER-GROUP", "args": {"action": "unlock", "n": 10}}
<class 'bytes'>


In [331]:
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')

ServerError: ERREUR 403, b'Salted__\x17g\x12l\x8b}\xc6$$p\x10\xec\xae\xde\xa2"\xbe\x88F\xc2\xb5\xd3\x10\x82Mxg3S\xfa\xfb\x8e~\xabx\x9c\xf2\x85\xfb[\xc1\x17\xdc\xe2\xac\xf6\x0eH\x1bk\xa7\xe5\xfel&\x8e[\xd0\x8fI\xd3N\x93\xf7\xf6A\xa1\xd5\x12H\x08f'

In [332]:
resultat = b'Salted__\x17g\x12l\x8b}\xc6$$p\x10\xec\xae\xde\xa2"\xbe\x88F\xc2\xb5\xd3\x10\x82Mxg3S\xfa\xfb\x8e~\xabx\x9c\xf2\x85\xfb[\xc1\x17\xdc\xe2\xac\xf6\x0eH\x1bk\xa7\xe5\xfel&\x8e[\xd0\x8fI\xd3N\x93\xf7\xf6A\xa1\xd5\x12H\x08f'

In [334]:
#resultat = base64.b64encode(resultat).decode()
#with open("reponse.txt", "w") as f: 
#    f.write(resultat)
reponse = decrypt("reponse.txt", K)
print(reponse)

Your mission does not allow you to perform this action


In [17]:
def commande_stp(method, url, nom_fichier):
    requete = {'method': method, 'url': url}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        f.write(resultat)

In [335]:
commande_stp("GET", '/service/hardware/status', 'status.txt')

In [337]:
reponse = decrypt("status.txt", K)
print(reponse)

┌──────────────────────────────Server Room Layout──────────────────────────────┐
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ S │ S │ S │ S │ o │ S │ S │ S │ S │0    f│ X │ ! │ X │ S │ o │ o │ X │ S │ X │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│   │ S │ S │ S │ S │ S │ S │ S │ S │1    e│ S │ X │ S │ X │ S │ X │ S │ X │ S │ 
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ . │ X │ . │ X │ X │ X │ X │ X │ X │2    d│ X │ X │ X │ X │ . │ X │ X │ X │ X │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│ S │ S │ o │ S │ S │   │ S │ S │ S │3    v│ X │ X │ X │ X │ X │ X │ X │ X │   │ 
│                                   │      │                                   │ 
│                

In [25]:
def commande_stp(method, url, nom_fichier):
    requete = {'method': method, 'url': url}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        for i in range(0, len(resultat)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(resultat[i])
            else : 
                f.write(resultat[i])

In [26]:
def action_serveur(url, option, n, nom_fichier): 
    requete = {'method': 'POST', 'url': url, 'args': {'action': option, 'n': n}}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        for i in range(0, len(resultat)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(resultat[i])
            else : 
                f.write(resultat[i])

In [24]:
def action_stp(url, option, nom_fichier): 
    requete = {'method': 'POST', 'url': url, 'args': {'action': option}}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        for i in range(0, len(resultat)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(resultat[i])
            else : 
                f.write(resultat[i])

In [25]:
def urgence(url, option, nom_fichier): 
    requete = {'method': 'POST', 'url': url, 'args': {'reason': option}}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        for i in range(0, len(resultat)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(resultat[i])
            else : 
                f.write(resultat[i])

In [18]:
def decrypter_fichier(file_name, k): 
    reponse = decrypt(file_name, k)
    print(reponse)

In [341]:
action_stp("/service/hardware/action/DOOR", "unlock", "ouverture_porte")

In [344]:
decrypter_fichier("ouverture_porte", K)

DOOR unlocked


In [346]:
action_stp("/service/hardware/action/LIGHT", "on", "ouverture_lumiere")

In [348]:
decrypter_fichier("ouverture_lumiere", K)

LIGHT on


In [354]:
commande_stp("GET", '/service/hardware/status', 'status.txt')

In [355]:
decrypter_fichier('status.txt', K)

┌──────────────────────────────Server Room Layout──────────────────────────────┐
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ S │ S │ S │ S │ o │ S │ S │ S │ S │0    f│ X │ ! │ X │ S │ o │ o │ X │ S │ X │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│   │ S │ S │ S │ S │ S │ S │ S │ S │1    e│ S │ X │ S │ X │ S │ X │ S │ X │ S │ 
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ . │ X │ . │ X │ X │ X │ X │ X │ X │2    d│ X │ X │ X │ X │ . │ X │ X │ X │ X │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│ S │ S │ o │ S │ S │   │ S │ S │ S │3    v│ X │ X │ X │ X │ X │ X │ X │ X │   │ 
│                                   │      │                                   │ 
│                

In [359]:
commande_stp("GET", '/service/hardware/action/SERVER-GROUP', 'commande_get.txt')

In [360]:
decrypter_fichier('commande_get.txt', K)

Possible values: lock / unlock.
Additional required argument: "n" (the number of the server group).


In [362]:
action_serveur('/service/hardware/action/SERVER-GROUP', 'unlock', 0xa, "ouverture_server_group")

b'Salted__\xb2\xf5#\xbd\xdf-\xd8\xf7\xb2K-k=\x923\x16p\xc7S\x05)\x15+O\xf9\xe8m\'\x9d\xd3.\x8c\x83\x87\xb8\xe5J+\xc4\xff\xfd\xfb\xb8\xe4\xdf9o\xca\x0bm\x19\xea\xac\x0f\xd8\x80\xef\xff)\xacC\xd1\xa6\xf873\xacD\x95U\xafdL\xca\x90\x95\x11<\xe0\xaa\tx\xe5X\xb1\x018\x1b\xc5]\xd5\xe9{"\xa5t\x87\x03\xf4h\xbbA\xb7\xec\x95\xdd(&G{\xa9\x19\x0e\xbe-\x00c{\xd6?'


In [364]:
decrypter_fichier("ouverture_server_group", K)

SERVER GROUP 0xa unlocked


In [407]:
commande_stp("GET", '/service/hardware/status', 'status.txt')
decrypter_fichier('status.txt', K)

┌──────────────────────────────Server Room Layout──────────────────────────────┐
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ o │ o │ o │ o │ o │ o │ o │ o │ o │0    f│ o │ o │ o │ o │ o │ o │ o │ o │ o │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│   │ o │ o │ o │ o │ o │ o │ o │ o │1    e│ o │ o │ o │ o │ o │ o │ o │ o │ o │ 
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ . │ o │ . │ o │ o │ o │ o │ o │ o │2    d│ o │ o │ o │ o │ . │ o │ o │ o │ o │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤m     ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│ o │ o │ o │ o │ o │   │ o │ o │ o │3    v│ o │ o │ o │ o │ o │ o │ o │ o │   │ 
│                                   │      │                                   │ 
│                

In [374]:
commande_stp("GET", '/service/hardware/action/SERVER', 'commande_get.txt')
decrypter_fichier('commande_get.txt', K)

Possible values: quick-test / deep-test / on / off / reboot. 
Additional required argument: "n" (the number of the server).


In [382]:
commande_stp("GET", '/service/hardware/911', 'urgence.txt')
decrypter_fichier('urgence.txt', K)

This is the server-room emergency response service. In case of emergency,
the data-center is shut down and all personnel is evacuated.

Because this interrupts the operation of the data-center, the emergency 
procedure must only be triggered when strictly necessary. All abuse will be
prosecuted.

To trigger the emergency procedure, send a POST request to this adress, with a
"reason" argument. The possible <reason> are:
    DDOS, FIRE, MAD-TECHNICIAN, ROGUE-IA, SHORT-CIRCUIT, AC-FAILURE, HACK


In [391]:
urgence('/service/hardware/911', 'MAD-TECHNICIAN', 'urgence.txt')

In [392]:
decrypter_fichier('urgence.txt', K)

Emergency Acknowledged.


In [397]:
action_serveur('/service/hardware/action/SERVER-GROUP', 'manual', 0xa, "ouverture_server_group")

In [398]:
decrypter_fichier('ouverture_server_group', K)

SERVER GROUP 0xa in MANUAL mode.


In [401]:
action_stp('/service/hardware/action/LIGHT', 'emergency', "urgence_lumiere")
decrypter_fichier('urgence_lumiere', K)VOLTAGE

EMERGENCY LIGHT.


In [405]:
action_stp('/service/hardware/action/VOLTAGE', 50, "urgence_v")
decrypter_fichier('urgence_v', K)

ServerError: ERREUR 403, b'Salted__a\xeb\x8d\x1c\x1a\n\xd4`\xa2\xad\xb0\x0fW\xd8c\xb6\xaf&\x12\xda<\x0c\x14{*\xcd\xb4{\x04\xa2Hh\x86\xcf\x14L>Y\xc4\x9b\xf5\xa4\xdf}\x10+s\xa6\x92*@\xe9\x03\x9d4\xbfn\x08\xfaI\xcf\x85\xd0\x0b\x14\x13\xd7\x84G\xa0\xf7t'

In [28]:
def action_stp(url, option, nom_fichier): 
    requete = {'method': 'POST', 'url': url, 'args': {'action': option}}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        f.write(resultat)

In [224]:
reponse = decrypt("LIGHT.txt", K)
print(reponse)

This is the server-room emergency response service. In case of emergency,
the data-center is shut down and all personnel is evacuated.

Because this interrupts the operation of the data-center, the emergency 
procedure must only be triggered when strictly necessary. All abuse will be
prosecuted.

To trigger the emergency procedure, send a POST request to this adress, with a
"reason" argument. The possible <reason> are:
    DDOS, FIRE, MAD-TECHNICIAN, ROGUE-IA, SHORT-CIRCUIT, AC-FAILURE, HACK


In [273]:
action_stp("/service/hardware/911", "FIRE", "ouverture_lumiere.txt")

ServerError: ERREUR 400, b'Salted__Z\xd5\xcd\tN\xf9SW\xe6\xca\x10\xdaNQ\xa8\r\xe8|\x97\xf5q_\x80\xc8\xf0\x82\t\xaa\xbd\x0c\x02pUW\x88\x03x\x00\xb2\x8e\xc0\xb8\xdf\xdf\xd9:i\x05B\xc1\xf2\x06\xee\x00%\xc4'

In [223]:
commande_stp("GET", "/service/hardware/911", "LIGHT.txt")

In [187]:
reponse = decrypt("ouverture_lumiere.txt", K)
print(reponse)

LIGHT on


In [ ]:
requete = {'method': 'POST', 'url': url, 'args': {'reason': option}}
requete_json = json.dumps(requete)
requete_chiffre = encrypt2(requete_json, K)
print(requete_chiffre)
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
resultat = base64.b64encode(resultat).decode()
with open(nom_fichier, 'w') as f: 
    f.write(resultat)

In [317]:
commande_stp("GET", '/service/hardware/status', 'status.txt')

ServerError: ERREUR 405, Method not allowed. You sent a POST method to /bin/gateway, which does not handle it.

In [268]:
reponse = decrypt("status.txt", K)
print(reponse)

┌──────────────────────────────Server Room Layout──────────────────────────────┐
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ ! │ ! │ ! │ ! │ o │ ! │ ! │ ! │ ! │0    f│ ! │ ! │ ! │ ! │ o │ o │ ! │ ! │ ! │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│   │ ! │ ! │ ! │ ! │ ! │ ! │ ! │ ! │1    e│ ! │ ! │ ! │ ! │ ! │ ! │ ! │ ! │ ! │ 
│                                   │      │                                   │ 
│                                   │      │                                   │ 
│ . │ ! │ . │ ! │ ! │ ! │ ! │ ! │ ! │2    d│ ! │ ! │ ! │ ! │ . │ ! │ ! │ ! │ ! │ 
│───┼───┼───┼───┼───┼───┼───┼───┼───┤      ├───┼───┼───┼───┼───┼───┼───┼───┼───│ 
│ ! │ ! │ o │ ! │ ! │   │ ! │ ! │ ! │3    v│ ! │ ! │ ! │ ! │ ! │ ! │ ! │ ! │   │ 
│                                   │      │                                   │ 
│                

In [295]:
commande_stp("GET", "/service/hardware/action/SERVER-GROUP", "SERVER_GROUP.txt")

In [297]:
reponse = decrypt("SERVER_GROUP.txt", K)
print(reponse)

Possible values: lock / unlock.
Additional required argument: "n" (the number of the server group).


In [281]:
action_serveur("/service/hardware/911", "MAD-TECHNICIAN","SERVER_GROUP.txt")

b'Salted__\xbdV\xa5\x08\x03\xa0\xcd\xbf\x1d\x14P\x89\xb6\xa8\xefK\xe6ny\x9c\xad%\xa6SR\x89\xe9\x7fu\xf6\xf0\x0cc\xbd\x8e0\x93\x8f\xcb\xa9Q\x8e\xba-\xbas\x1f\xads\xa77\x89%\xab\xaa\x9c8(\xa4T6\x90ed!\x9c\x04\xe8@^\xb9\xbe\xa5WSa\x0b\xf9q\x8a\xed\x938Q\xc6\x94\xca\x85t\x82\xb3\xe8 s\x03\xdcg\xdb\xadA1\x805{'


ServerError: ERREUR 403, b'Salted__\xa9B\xfc\x10}{\xd6\x8doL\x16;\xa4\xa1s1V-\xa1|\xc28D\xf0]\xd6\x1a\x8e\n\xb5\x7f\x02Vu\x08A`\xd6\x7f\xda\x84\x1c:\x99\x81O\xc73_\xab\xf8\xbdb\xe0\xeeD\xea\xdc\xf6\xa8\x8a\xe43!\x86\xc7\xbe\xcfu\x89\xa3\xc5'

In [282]:
resultat = b'Salted__\xa9B\xfc\x10}{\xd6\x8doL\x16;\xa4\xa1s1V-\xa1|\xc28D\xf0]\xd6\x1a\x8e\n\xb5\x7f\x02Vu\x08A`\xd6\x7f\xda\x84\x1c:\x99\x81O\xc73_\xab\xf8\xbdb\xe0\xeeD\xea\xdc\xf6\xa8\x8a\xe43!\x86\xc7\xbe\xcfu\x89\xa3\xc5'


In [284]:
#resultat = base64.b64encode(resultat).decode()
#with open("serveur.txt", 'w') as f: 
#    f.write(resultat)
reponse = decrypt("serveur.txt", K)
print(reponse)

Emergency denied. No reason to do so. You will be sued!


In [269]:
action_stp("/service/hardware/action/DOOR", "unlock", "ouverture.txt")

ServerError: ERREUR 403, b"Salted__\x8c\xc8\xdc8vQ\x0f\xc1\x0f\xf2?6\x1e\x05Y\xb1V\xe2s{\x05\xc7\x07}Z\x93v\xcb\x916\x9e\xd8\x93x#=\xa7\\*\ti6\x91)\x12'S\xf1f,\n\xebF\x14\x9cxP1\x14\xbc\x08\xd0\xd3\xf3W\x18\x8a\xe3\xcfk\x19\x8a"

In [265]:
reponse = decrypt("ouverture.txt", K)
print(reponse)

AIR CONDITIONNING on


In [276]:
requete = {'method': 'POST', 'url': "/service/hardware/911", 'args': {'reason': "FIRE"}}
requete_json = json.dumps(requete)
requete_chiffre = encrypt2(requete_json, K)
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
resultat = base64.b64encode(resultat).decode()
with open("ll.txt", 'w') as f: 
    f.write(resultat)

ServerError: ERREUR 403, b'Salted__\x8cUL\n\xa2\xe9s3\x8b\xb1\xc0-\xa8\xfe!{\xff\x93C\x8c\xcbxk\xe2Q($\xcf\x1b?\\[H\xc9"\xd7 r\x1c]\x81\xa5\xc9\xfdkY\xae\xff\xef\x89\xe9;\x8d\xb2m\xb5A\x04s\x82\xd9|G\xea\x8b;",\xcb1d\xa3'

In [312]:
requete = {'method': 'POST', 'url': "/service/hardware/action/SERVER-GROUP", 'args': {'action': "unlock", "n": 0xa}}
requete_json = json.dumps(requete)
requete_chiffre = encrypt2(requete_json, K)

In [313]:
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')


ServerError: ERREUR 403, b'Salted__\xaetvmu\xbf{\xf7|C\xb4\x94\xfb5\x00lT\xd9\x9a\xea\xb1T\xe2\x0b\xeb\xdc\x99\xe4\xfdpe\x11\xe8f\x17\xb9\x9aK\xe1\xd9E\x8d\xae\xfa\x0eElf\x08\xf2Da:\xa6\xf6a\xf9\xbd\x16\xb4m\xcfL\xaa\x0b\x02%\x0f\xb1`\xca\xa4'

In [314]:
reponse =  b'Salted__\xaetvmu\xbf{\xf7|C\xb4\x94\xfb5\x00lT\xd9\x9a\xea\xb1T\xe2\x0b\xeb\xdc\x99\xe4\xfdpe\x11\xe8f\x17\xb9\x9aK\xe1\xd9E\x8d\xae\xfa\x0eElf\x08\xf2Da:\xa6\xf6a\xf9\xbd\x16\xb4m\xcfL\xaa\x0b\x02%\x0f\xb1`\xca\xa4'


In [315]:
resultat = base64.b64encode(reponse).decode()
with open("ll.txt", 'w') as f: 
    f.write(resultat)

In [316]:
reponse = decrypt("ll.txt", K)
print(reponse)

SERVER GROUP 0xa unlock: CANCELLED (OVERRIDE engaged).


In [22]:
print(c.post('/bin/police_hq/ticket/1174/close'))

"STATUS : OK --- ticket 1174 closed\n\nDEBRIEFING\n----------\n\nL'intervention du technicien est terminée. Le technicien a été victime d'un\nincident d'exploitation.\n\nLe rapport d'incident vous met hors de cause.\n\nAu cours de l'intervention, un programme système a eu un comportement inattendu.\nNous transmettons un rapport aux administrateurs UGLIX. Nous notons que ce genre\nde problèmes semble se multiplier.\n\nCONCLUSION: Mission terminée. \nISSUE : transmission d'un bug-report à root.\n\n[ You have new mail ]"

In [19]:
nonce = c.post("/bin/login/stp", username= "palmerjohn")

In [17]:
K = "debug-me"

In [18]:
text = "Väinö"
text = text.encode()
text = base64.b64encode(text)
print(text)
dico = {'method': 'PUT', 'url': '/bin/echo', 'data': 'VsOkaW7Dtg=='}
requete_json = json.dumps(dico)
requete_chiffre = encrypt2(requete_json, K)

b'VsOkaW7Dtg=='


In [19]:
resultat = c.post_raw(url = '/bin/test-gateway', data = requete_chiffre, content_type='application/octet-stream')

In [20]:
print(resultat)

b'Salted__\\\xf9<\xc1^\xe4D8\x0bR\xdc,\x12\xd6\xff\xac\xf2\xd4\xff\x97\x91\xaf\xa2\x19d\xd7#\x08\xc4\x7f\x94%R\x80\xfa\xf2\xbcM@\xad1i \xa9\xcc\x9aU#\x02\x18\x8dy\x15gw>\xc8r\x02}\xb8_T1\xe0\x19hW\xae\xdd\xe6Ax\xe5\xfe\x99\xf6a\xbdq\xca\x8d\\@7v]\xc1\x15.\xb3{s\x87K[\x0e\x0f\x95\x03\xd8\xaauk\x0c\xe1:\xadC\x04.\x91\xd6rY\xf8\x85\xce\xf9w\x8b\xe8\xc9\xde\xd5\xd6\xaa\rR\xf51\xda\xcf\x1e(+'


In [22]:
reponse = b'Salted__\\\xf9<\xc1^\xe4D8\x0bR\xdc,\x12\xd6\xff\xac\xf2\xd4\xff\x97\x91\xaf\xa2\x19d\xd7#\x08\xc4\x7f\x94%R\x80\xfa\xf2\xbcM@\xad1i \xa9\xcc\x9aU#\x02\x18\x8dy\x15gw>\xc8r\x02}\xb8_T1\xe0\x19hW\xae\xdd\xe6Ax\xe5\xfe\x99\xf6a\xbdq\xca\x8d\\@7v]\xc1\x15.\xb3{s\x87K[\x0e\x0f\x95\x03\xd8\xaauk\x0c\xe1:\xadC\x04.\x91\xd6rY\xf8\x85\xce\xf9w\x8b\xe8\xc9\xde\xd5\xd6\xaa\rR\xf51\xda\xcf\x1e(+'

In [27]:
resultat = base64.b64encode(reponse).decode()
with open("ll.txt", 'w') as f: 
    f.write(resultat)

In [29]:
reponse = decrypt("ll.txt", K)
print(reponse)

Sent: Väinö
[[1613216153##kernel-monitord[42] UGLIX]]: TEST.GATEWAY:14:3@578991|1f879edf250a709c5f656122549b098d


In [102]:
print(c.post('/bin/police_hq/ticket/1175/close'))

STATUS : OK --- ticket 1175 closed
[ You have new mail ]


In [22]:
print(c.get('/bin/police_hq/ticket/1175'))

Un de nos collaborateurs, michael60 vient de subir un regrettable accident
et se trouve actuellement dans un état végétatif. D'après notre équipe médicale, 
il y a peu d'espoir de voir son état s'améliorer. Il était stagiaire dans 
l'équipe des admins de notre agence.

Une enquête préliminaire, dirigée par [[---REDACTED---]] semble se diriger vers
l'hypothèse qu'il ait tenté de mettre fin à ses jours, pour des raisons inconnues.
Il était l'une de nos jeunes pousses les plus prometteuses, et toute l'agence
regrette profondément la tournure tragique des évenements.

Par une étrange coincidence, michael60 faisait l'objet d'une enquête discrète, 
suite à la découverte de son éventuelle implication dans une association 
subversive.

Malheureusement, nous ne pouvons plus accéder à son compte. Nous espérions y
trouver des informations qui pourraient expliquer son geste d'un part, et 
poursuivre l'enquête en cours sur ses activités.

Cependant, nous avons pu obtenir (en utilisant [[---REDACTED

In [23]:
words = c.get("/share/words")
liste_de_mot = words.split('\n')

In [19]:
def decrypt4(filename, password, cipher ='aes-128-cbc'):
    pass_arg = 'pass:{0}'.format(password)
    args = ['openssl', 'enc', '-d', '-pbkdf2', '-' + cipher, '-base64', '-pass', pass_arg, '-in', filename]

    if isinstance(filename, str):
        filename = filename.encode('utf-8')

    result = subprocess.run(args, input=filename, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    error_message = result.stderr.decode()

    if error_message != '':
        raise OpensslError(error_message)

    res = base64.b64encode(result.stdout).decode()
    return res

In [20]:
def test_mot_de_passe(liste, plan_text, reponse):  
    res =[]
    for i in (liste):
        try:
            resultat = decrypt4(reponse,i)
            print(i)
            res.append(i)
        except OpensslError: 
            pass 
    return res

In [21]:
plan_text = 'michael60-7a22aade010f4d9b92f3b612be0127ec'
reponse =  "U2FsdGVkX18JuKl7lv06NQ+X4o3N4Ok+zTwkf/C1Ox7LToL4NUoUImBNVBFqVkIo\nRaDGFn3lvJDpD1fSYeWqFw==\n"

In [30]:
mdp = test_mot_de_passe(liste_de_mot, plan_text, "reponse.txt")
print(mdp)

accentuates
acknowledgment
activated
aggregately
albeit
Aldrich
aliased
aloes
analyzers
anesthetized
apathetic
appearing
artisan
ashtray
aspirin
assassination
assurer
attainment
aviary
barrenness
bashes
begrudged
Belleville
beribboned
bloated
blueberry
bolts
bombast
bonded
bray
bridging
bullying
bypassing
Carey
ceremonially
certify
chartings
cluck
cohering
conformity
congest
connotative
consigning
cop
Cuban
dangled
decimal
deride
deserter
dictatorial
din
discover
dozes
drilled
Dubhe
dungeon
eager
effects
eligibility
emphatic
enterprising
envelopes
epitomizing
evictions
exhortations
expressed
extraordinariness
Exxon
fancier
Fidel
fiefdom
flattening
forewarnings
fraught
galloped
Garrisonian
gash
Gaston
godlike
good
Grayson
greenware
hello
heralding
Homeric
ignition
impersonal
incomplete
incurs
inhibitory
initiated
intelligence
interlacing
internalizes
jested
Jules
junk
kerchief
kidnappings
labor
lace
lain
lames
leaderships
marigold
massacres
milder
millivoltmeter
minuteman
monotonically


In [32]:
for b in mdp: 
    try:
        print(c.post('/bin/login', user= 'michael60', password= b))
        print(b)
    except ServerError: 
        pass

PAP login successful. Access to /home/michael60 granted.
[ You have new mail ]
violates


In [124]:
print(base64.b64decode(b'0x90'))
print(str(b'0x90'))

b'\xd3\x1ft'
b'0x90'


In [28]:
username = c.get('/bin/police_hq/ticket/1175/attachment/username')
challenge =  "8e32e4f623b043bab250d0e061f6c427"
plan_text = "michael60-cf7032dc716c4a10ba83cb35aad595b1"
reponse =  "U2FsdGVkX19ok25XZNj9LC7fsTU3TJs4/VoP2CXpCJFuM7UNXjuJ1ue4/ahzeSy1\ntMJ0VqikDHk1GrGimvtImg=="
trace = c.get('/bin/police_hq/ticket/1175/attachment/trace')

print("username = {}".format(username))
print("trace = {}".format(trace))

username = michael60
trace = > GET /uglix/bin/login/CHAP HTTP/1.1
> Host: UGLIX.HAL.DiscoveryOne
> User-Agent: community-script v2.1 [h4ck3d by ]
> Accept: */*



< HTTP/1.0 200 OK
< Date: Sat, 22 Aug 21887 13:05:21 GMT
< Server: UGLIX Login Program [challenge]
< Content-Type: application/json
< Set-Cookie:  sessionid=79c86tw9zwkmqlcs58ie8ukto3lxuc0v
<
{"challenge": "19c4a3e2b8f24cd493a1dd40e83e5008"}



> POST /uglix/bin/login/CHAP HTTP/1.1
> Host: UGLIX.HAL.DiscoveryOne
> User-Agent: community-script v2.1 [h4ck3d by ]
> Accept: */*
> Cookie: sessionid=79c86tw9zwkmqlcs58ie8ukto3lxuc0v
> Content-type: application/json
> Content-Length: 147
>
{"version": 2, "username": "michael60", "response": "U2FsdGVkX18W5HQgvZ91/M1rf+r5FiqE0cqxYOAIDFB8YtgW7JScuEUvFVyj1lJ9\nFVF+ajRARoMthnSJkwfZcA==\n"}



< HTTP/1.0 200 OK
< Date: Sat, 22 Aug 21887 13:05:24 GMT
< Server: UGLIX Login Program [main]
< Content-Type: text/plain
< Set-Cookie:  sessionid=0i1voxkain7neemy822he7jqxgkkobom
< 
Login successful.

In [92]:
username = c.get('/bin/police_hq/ticket/1175/attachment/client')
client = username['email']

In [99]:
c.post('/bin/sendmail', to=client, subject="mdp", content="violates")

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [47]:
c.post('/bin/login', user= "michael60", password= 'violates')

'PAP login successful. Access to /home/michael60 granted.\n[ You have new mail ]'

In [51]:
reponse = (c.get('/home/michael60'))
print(reponse)

# Bienvenue, michael60.
# Dernière connection le 17-02-21893 14:24:42 (protocole : PAP)
# Vous avez 2 messages non lus (4 en tout).

Rumeur du jour: 

Les comptes de certains vieux programmeurs UGLIX n'auraient pas été supprimés après leur départ.

***** Contenu de votre dossier personnel ******

  /home/michael60/TODO
  /home/michael60/md5_collider.tar.gz
  /home/michael60/research.jpg
  /home/michael60/chat.log



In [52]:
md5 = c.get('/home/michael60/md5_collider.tar.gz')

In [53]:
s2file(md5, "md5_collider.tar.gz", "wb")

In [65]:
reponse = b"A) FSCK\n=======\n\ninjecter collision\nv\xc3\xa9rifier syst\xc3\xa8me de fichier (/bin/fsck)\n---> bug\nsuivre directives\n\nCollision ?\n\n* MD5(A) == MD5(B) == MD5(C) == MD5(D) and len({A,B,C,D}) == 4\n\n ^\n/!\\     Tous les fichiers commencent par le nom d'utilisateur UGLIX\n---     Tous les fichiers finissent par 'h4ck m0d3'\n |\n\nSolution la plus simple?\n\n\nB) STOCKAGE P\xc3\x89RENNE\n===================\n\nC'est quoi le probl\xc3\xa8me avec la bande magn\xc3\xa9tique num\xc3\xa9ro 4919 ? Pourquoi l'op\xc3\xa9rateur\nsort une excuse bidon ?"
reponse = decrypt(reponse, 'violates')

OpensslError: Can't open A) FSCK
=======

injecter collision
vérifier système de fichier (/bin/fsck)
---> bug
suivre directives

Collision ?

* MD5(A) == MD5(B) == MD5(C) == MD5(D) and len({A,B,C,D}) == 4

 ^
/!\     Tous les fichiers commencent par le nom d'utilisateur UGLIX
---     Tous les fichiers finissent par 'h4ck m0d3'
 |

Solution la plus simple?


B) STOCKAGE PÉRENNE
===================

C'est quoi le problème avec la bande magnétique numéro 4919 ? Pourquoi l'opérateur
sort une excuse bidon ? for reading, No such file or directory
140541089768640:error:02001002:system library:fopen:No such file or directory:crypto/bio/bss_file.c:69:fopen('A) FSCK
=======

injecter collision
vérifier système de fichier (/bin/fsck)
---> bug
suivre directives

Collision ?

* MD5(A) == MD5(B) == MD5(C) == MD5(D) and len({A,B,C,D}) == 4

 ^
/!\     Tous les fichiers commencent par le nom d'utilisateur UGLIX
---     Tous les fichiers finissent par 'h4ck m0d3'
 |

Solution la plus simple?


B) STOCK

In [37]:
resultat = c.get("/bin/fsck")
print(resultat)

fsck from UGLIX-util 2.20.1
ufsck 1.42 (29-Nov-18011)
/dev/flash0 contains a (clean) UglFS file system, manual check forced.
Pass 1 : Checking unodes, blocks, and sizes
Pass 2 : Checking (blocks vs files) maximum-flow
Pass 3 : Checking that block subgraph contains a (large) independent set
Pass 4 : Checking longest cycle in file subgraph
Pass 5 : Checking 3-colorability of dual file graph
Pass 6 : Checking for file deduplication issues
fsck completed successfully.


In [48]:
c.post('/bin/sendmail', to='anonymous.coward', subject="A", content="Hey")

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [50]:
mail("palmerjohn", '13323')

----------------------------------------------------------------------
Message #13323.


Date: 13-02-21893 12:15:16
From: anonymous.coward
Subject: Re: A

Salut palmerjohn,

Je suis le programme écrit par anonymous.coward pour gérer sa boite mail.
Malheureusement, je ne peux pas transmettre ton message à mon créateur.
Ce n'est pas personnel, j'ai été conçu pour ça.

Mon créateur tient à prendre certaines précautions pour sa propre sécurité.
Il tient à te faire savoir que, par conséquent, si jamais tu veux entrer en contact
avec lui, il te faut montrer patte blanche. Pour cela, il faut que tu aies une
clef publique signée par (au moins) trois autres utilisateurs. Pour l'instant,
ta clef publique n'est pas signée (cf /doc/web-of-trust)

a+,
-----
mail-filterd

PS : s'il répond, mon créateur envoie des mails chiffrés avec la technique du
     chiffrement hybride.




In [41]:
def decrypter_fichier(file_name, k): 
    reponse = decrypt(file_name, k)
    print(reponse)

In [22]:
def signature(document, secret_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'dgst', '-sha256', '-sign', secret_key]
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(document, str):
        document = document.encode('utf-8')
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=document, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return base64.b64encode(result.stdout).decode()

In [20]:
key_thomas = c.get('/bin/key-management/melissaanthony/pk')

In [21]:
key_thomas_signe = signature(key_thomas, 'key_private.pub' )

In [22]:
print(type(key_thomas_signe))

<class 'str'>


In [35]:
key_ami = c.get('/bin/key-management/wnunez/pk')
key_ami_signe = signature(key_ami, 'key_private.pub' )
print((key_ami_signe))

TwrTTN6FDzKf9EWanWwyLrzqHCaH2MMc6rOS/ZawBQF0mv5BMKNuDzcRqg4Y/y0sesafmg0tKW+tgmjeumX1pyAD29I7TUykTw4wh4VCTmDM5mh51ed8K95H7rVYS3KGgSrSJ2PLEQFgaH+1cK6YQ0RJFMnb7MHU2FPtYsDJ6NhKfeYJSRneo4cW9SIfJpDJiGTZtgXSYLnYpcB20qBkI3umstEtxhDIWdGqIN9QPly9Wd4NR3jEagVdsJKs+8dWquY+kH1uMAOcYBl6RnJ8KeMPqj9fWOwSg0Bfs2VuM59A1uyoWchCf1V6CsqerjFztquC7uWI3HfGKD47Btt/bQ==


In [36]:
c.post('/bin/sendmail', to='wnunez', subject="Le Boss", content=key_ami_signe)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [262]:
key_Quentin = c.get('/bin/key-management/debradavidson/pk')
print(key_Quentin)

-----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQC7VrEmUqoamceLQzW6Q6vCwAEN
aJgbeUVkBiCYHO+m3LdKNUmzoM88UxqDKYBgHVjvF8YJtgMihjU6GAD07Ywgm3+j
skQGOY60Y816Y5unSS6a21isBEENCtqAX6iiV/QGms8PrP2zG5lJq7lda1swljGH
T4uYuGh8wC2Jev95uwIDAQAB
-----END PUBLIC KEY-----



In [263]:
key_Quentin_signe= signature(key_Quentin, 'key_private.pub' )


In [264]:
print(key_Quentin_signe)

kqA2ZWKKLe/FlWhG5MLtX2yB6Io43QroFB8NCrr+xsaXt8P8KshcxSE4dc2dJWz9rOtxPbSElx2J5GzbrKG1kaMBJEPwI7ntk5FV7ccj+GhPIAe3GS1kNusM+ZYFfeos9i8UTEy6Dg9LVs4lF2KcirHMUslbS0fTCwEgnw/XHmbGB/OZEKL43HMREHM7q1GhXLDKjq6ItE++nI0fCgeM6owfa9UCcshu5IeBX1CQiX6Q/fPX+o9lCH2nfyDprOI/4LzWcfoTbo9VmGWcGIXdnmQTHg5fRob+JJIVEhl84+lh+GUiKM+ANqCIqrVitzsTN61m9EQNMCFO+eH13qgdDA==


In [265]:
c.post('/bin/sendmail', to='debradavidson', subject="A", content=key_Quentin_signe)

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [48]:
Ma_cle_signe_par_quentin = mail("palmerjohn", "13433")

----------------------------------------------------------------------
Message #13433.


Date: 13-02-21893 18:23:22
From: debradavidson
Subject: signature

TOLjj8lD2E04ATA6PV/CvOCcghfCnTgvQRbx3fVQ07t/d1yWuWNjegenabbJez0NPCGhxcOGQeEkEaKVI/lzX13AbQ0NiSJhWEmCbT7AhFNkuZnmeD12KNxYi/8P3OCxSXkvkXc1nenIby/Jx6XDVnTZ30Rki0UP/q9gCrga4P8=




In [49]:
quentin = 'TOLjj8lD2E04ATA6PV/CvOCcghfCnTgvQRbx3fVQ07t/d1yWuWNjegenabbJez0NPCGhxcOGQeEkEaKVI/lzX13AbQ0NiSJhWEmCbT7AhFNkuZnmeD12KNxYi/8P3OCxSXkvkXc1nenIby/Jx6XDVnTZ30Rki0UP/q9gCrga4P8='

In [11]:
Ma_cle_signe_par_thomas = mail("palmerjohn", "13341")

gbkO8TZE1da7jkX8iyDkKRm0IlRYjLNvBa3MpKf7UZEZAKtj0Vq5Xas7/CMd92bL09IuU4UA7XD1+bnduw0I0T1uIDvXF3n7vqDEYGc49Ma6a+N9PjpUBjNyx8jNrpfbdOEzpfDZcShGnQJydf8HIqzog301E3iN2x1TOMeXZPF6QYe+qh3WwCLNBGCPDT7/0WjBipkmHNVPEL64TKxB08PE6GpOEXH0Cv+tAj78bU1XHYoYX/BgR2PJwZySyjzVQDxuihDKE2/ibpd/EhneqBkN6U0GBIjgf4eo6F9WQVTsGt/z5b6+j17ZYHZMF76OJqTJMYqRt0sVB+hf/GqofQ==


In [29]:
thomas = 'gbkO8TZE1da7jkX8iyDkKRm0IlRYjLNvBa3MpKf7UZEZAKtj0Vq5Xas7/CMd92bL09IuU4UA7XD1+bnduw0I0T1uIDvXF3n7vqDEYGc49Ma6a+N9PjpUBjNyx8jNrpfbdOEzpfDZcShGnQJydf8HIqzog301E3iN2x1TOMeXZPF6QYe+qh3WwCLNBGCPDT7/0WjBipkmHNVPEL64TKxB08PE6GpOEXH0Cv+tAj78bU1XHYoYX/BgR2PJwZySyjzVQDxuihDKE2/ibpd/EhneqBkN6U0GBIjgf4eo6F9WQVTsGt/z5b6+j17ZYHZMF76OJqTJMYqRt0sVB+hf/GqofQ=='

In [30]:
c.post("/bin/key-management/upload-signature", signature= thomas, signer= "melissaanthony")

'OK'

In [50]:
c.post("/bin/key-management/upload-signature", signature= quentin, signer= "debradavidson")

'OK'

In [43]:
with open('graph.pdf', 'wb') as f:
    f.write(c.get('/bin/key-management/trust-graph'))

In [47]:
mail("palmerjohn")

----------------------------------------------------------------------
INBOX: There are more than 25 messages. Only displaying the 25 last messages.

Message | Read | Sender                   | Subject
--------+------+--------------------------+---------------------------
 13433  |   N  | debradavidson            | signature
 13428  |   Y  | anonymous.coward         | Re: A
 13422  |   Y  | wnunez                   | Envoie signature 
 13348  |   Y  | yfrench                  | Re: mdp
 13347  |   Y  | yfrench                  | Re: mdp
 13341  |   Y  | melissaanthony           | signature de ta pk
 13339  |   Y  | debradavidson            | signature
 13325  |   Y  | melissaanthony           | Hey Moha
 13323  |   Y  | anonymous.coward         | Re: A
 13321  |   Y  | anonymous.coward         | Il faut qu'on parle
 13320  |   N  | anonymous.coward         | Il faut qu'on parle
 13319  |   N  | anonymous.coward         | Il faut qu'on parle
 13318  |   N  | anonymous.coward         | I

In [40]:
Ma_cle_signe_par_ami= mail("palmerjohn", "13422")
print(Ma_cle_signe_par_ami)

----------------------------------------------------------------------
Message #13422.


Date: 13-02-21893 18:11:48
From: wnunez
Subject: Envoie signature 

XPktvGI2paiVQM38J553byimM7ySQVwUVII4TecY448s/cmSvxZlD4ROIUm3EAaaS3felwIEytIgkKWNUiL/PaNjwl8qdKUcynCW+z16UiP0wakYqMMuMATvqoUQWd8/gFCoYsLGZnAK04A+SQ7J9b+SaqDdsROsnyAefz+Ld+Q=


None


In [41]:
Ma_cle_signe_par_ami ='XPktvGI2paiVQM38J553byimM7ySQVwUVII4TecY448s/cmSvxZlD4ROIUm3EAaaS3felwIEytIgkKWNUiL/PaNjwl8qdKUcynCW+z16UiP0wakYqMMuMATvqoUQWd8/gFCoYsLGZnAK04A+SQ7J9b+SaqDdsROsnyAefz+Ld+Q='

In [42]:
c.post("/bin/key-management/upload-signature", signature= Ma_cle_signe_par_ami, signer= "wnunez")

'OK'

In [51]:
c.post('/bin/sendmail', to='anonymous.coward', subject="A", content="FINI")

'STATUS: OK --- mail delivered succesfully\n[ You have new mail ]'

In [80]:
resultat = mail("palmerjohn", "13441", "/body")

{"session_key": "p/KagaJyMF10HludIZeIIWHKnLNbeJURhK1iiI5zWX4Y8h9cD/lpPESj1gLwnYwaDynkq60cEEi18QEHqvdYrn3wLs/0N5jGCdCxkJdYdrTrnBH8aqYqW2NOtdDPPeW7VgR8jOiSbL43fLNRlqypGF+ZyFaNe/NkXMZMAIo0vWY282sVdbQ2Cun+vttbWB5OjQQ3BOW3SsdhA11pC5fI/djCcvASfoUiCjGNf2/Ln61cBSDtFsoVd7HOX81ZKL5zCA8S1Zs2jAMQBCjRJHaLGF8sMX6aIqyJ8pX9v5EDQLWbyBlkTIdbNyT5VlbK5HMhXXFsDiyQllsnX492hGPJ+Q==", "payload": "U2FsdGVkX1+M39RPH0Qpu4jkfT07T5gF6T0QF6KpLGGPjdUJpokWcb806bBTtfaP\n055tJ3XY5H/TRfxOoxu/40q+tCjouXpAIj9PmMCCo26XubQpEADqmhvrkJCZslSA\nhsmiuWiHbwEk8Ecf6PR86UTZxW87cOG76IKDuh65u8Xoi2HyrSOpsSwc7JfNx/R6\npAcJ7cT9ZvvDUhVG74ALMgJjT1XnI8MuZBMHxwPGHNq6BZ/8wgugU4tjhMgt5NYb\nHb28nLYdNaCGF872LmpgzuUg00kyc2h4Te8N57f9qPno+9O0vwVy5iSzvZfGDPyT\nl1GI71Bt4FKkzhSyJcFb2DXxVXGQ8y/FDyv8jJiLGb/CukvRaARtMLC1loLhN+pQ\nj+YFar/SVZQ15h23Tar/STOkBkLNVZQGyfatdij1HjjVkv2hJcsAaznKu0SUN2oT\n6vOHyK6dfnvIShCf66KxDOROUrPXuxZyrW6CCKTII2MqxSXObDmN8aZ84EsKRB+4\nFfzsTRE96SMI95YYGOoje19pw/cWLyZYPf3qtz8X912AZ0EP8yilWMM7jvmQeDHg\nGexRzA5q2/TaoGDMCfg0XkSdTeRb7L

In [13]:
resultat ={"session_key": "p/KagaJyMF10HludIZeIIWHKnLNbeJURhK1iiI5zWX4Y8h9cD/lpPESj1gLwnYwaDynkq60cEEi18QEHqvdYrn3wLs/0N5jGCdCxkJdYdrTrnBH8aqYqW2NOtdDPPeW7VgR8jOiSbL43fLNRlqypGF+ZyFaNe/NkXMZMAIo0vWY282sVdbQ2Cun+vttbWB5OjQQ3BOW3SsdhA11pC5fI/djCcvASfoUiCjGNf2/Ln61cBSDtFsoVd7HOX81ZKL5zCA8S1Zs2jAMQBCjRJHaLGF8sMX6aIqyJ8pX9v5EDQLWbyBlkTIdbNyT5VlbK5HMhXXFsDiyQllsnX492hGPJ+Q==", "payload": "U2FsdGVkX1+M39RPH0Qpu4jkfT07T5gF6T0QF6KpLGGPjdUJpokWcb806bBTtfaP\n055tJ3XY5H/TRfxOoxu/40q+tCjouXpAIj9PmMCCo26XubQpEADqmhvrkJCZslSA\nhsmiuWiHbwEk8Ecf6PR86UTZxW87cOG76IKDuh65u8Xoi2HyrSOpsSwc7JfNx/R6\npAcJ7cT9ZvvDUhVG74ALMgJjT1XnI8MuZBMHxwPGHNq6BZ/8wgugU4tjhMgt5NYb\nHb28nLYdNaCGF872LmpgzuUg00kyc2h4Te8N57f9qPno+9O0vwVy5iSzvZfGDPyT\nl1GI71Bt4FKkzhSyJcFb2DXxVXGQ8y/FDyv8jJiLGb/CukvRaARtMLC1loLhN+pQ\nj+YFar/SVZQ15h23Tar/STOkBkLNVZQGyfatdij1HjjVkv2hJcsAaznKu0SUN2oT\n6vOHyK6dfnvIShCf66KxDOROUrPXuxZyrW6CCKTII2MqxSXObDmN8aZ84EsKRB+4\nFfzsTRE96SMI95YYGOoje19pw/cWLyZYPf3qtz8X912AZ0EP8yilWMM7jvmQeDHg\nGexRzA5q2/TaoGDMCfg0XkSdTeRb7L2iOAE2UavvvNudD2qwdTFpNbrVkwexX4Uh\nweM/zc5e+uDaBGDTrisIY2fbzIounnnKPw36/cVLCwFmKiBvwOtN6xStENV9bqSg\n/ioLaF/Ug4sfKLltugeIbg+pLeTyZeT2DaLvl8pCawmUQZkPZG68lSavEbUgKQ4B\n100BBNqOZ+E4uYeh/E5Heufx6EeEQuFxaCE4GLI2JmC1We2MMRLdprSWPecM+5Nl\nYkpy/wMboI3jdvxWJg7/JqEALT2jvvRVlHW1wMU3YO3dCCLA65AAfIHf8A7FRwX7\n5bCANdUSIxKueHa2BsiBDlHR0Et7gWRHV5ShsUOSwOEvj08FO+tWSEedhySG5Owd\nWM2nyK336AtJjrilbqhnFQQ6cGh7r9oT07jlTrf2A/lb79J+WIdt6gdPUK1q7FbO\nhBhwLpzYb4ked34Jk5jNTHJXH6yQGdEc/O8hZZTjNQoMSaFB9mRWHBSi6FHYAK8b\ntd4CsTHOAnYUrCOBydFHV85Gvr5um7byQTG1fJH4zWvCChutB/p3kV/Hw0EMabMn\n2AJpRFx+igXGj3KPQTwDk6eK60U5QP50+aWeJFN/uNoPbM3SyGEoJU/M/Kwbd564\nJgum8XwgsjMge1UuNNx+IEf8X0yFRS0TVO5X6Ef9lJY8MVKP+4hTendNnMp7JlaK\nf4UKe0UvgwSH/9A/EEQnObOsaxdDhBhegAxe5QWo4XoEJMSJWCzT6Q8SOMlozVMj\n0oQbsiGTSHor/YHcikFxKnvGUb8qsDTJTy4Xd1+w20VFyUG9+h4Tww93Wg0B4C30\n3PUT3I19HxpsPLhuyHnLhIyrWoBXEqsXZuFAx7ZNVGFp8wZsZTlO7RpkCvtib2JB\nIrn3xcfxVXzLCKstw8VIp62Zz5poxo2T3Mysksxf1ForwFQg7/6spUaJ8Mj5OCCJ\nSn7n44R9FXGmSHYVgYyG/F8TyR2fhYB8KZmNZcLdkZgm0sDvFI829YJzs34ct62T\nno/aoP2lvA7UfzCA3OI+c93fEpQpNMowFIu7pzvnoeoh+pc9DCh6D92WxaUn5sdI\n5/ybJrcm4WJpm7gxAxBhY/PNJsiaExjruwnJiZs1vfeAszdaoLnh88cxL95s2PBn\nYALYoyA7F1R6efq6NwUTUxARJ7PMUcrMUJc7/ZRF1J3oZA3R587eHI9BHrLpJApd\n8Rxpib16dEAPF1IskFJYPI/GazhotcV1iILNUUyirCX+gXf1MobgDRzLrbz6c4Dp\naiU8N5QXuHcU7NQ3i6v3U+Z4XJF11IHUIW2TEl6v4WWKrEyqrG9h+/mqId37OwgW\nqHE3tCww3uK+vDYxZfsqK8LXUro/7rf5daJqPvHifgdkQEhEu/eIaNAlhzqJ33WV\nOZMKhCfMrezz27VqTHb62AXEN9VA7hE3Jmdh84xVVuzfVN03CWtQQRPrQ3yu/KQw\ngmEtWJbGV12ouqQXzZaGM64L3X30SgfjX0DSPKRGO/udoJJr88MxH2IQAArzRQW1\nQ8+GyGXfuB872Cl0ar5EIbKWldNAw1JUyvsRsrsHmE0RVDlqqCOV8sZgHsfHF3yH\nQ4ZPbagXRJjpi/nV636MFmnGvuy38rr3bRq5rHhE+ENYnp3H+sWq/j2DyXgvTely\nsQFgK58/3Tz1uYrOdhxtu3dAXjsYmAPhsUZEFuWiiWLCRgPwSjdewQsL0O7nNCtm\nHF/X8rjGFE0kO23hc945PHvowe4tU9Ob5AvHCTu8WG37GQFPZimGz5MS3Rh6y6pG\npbSA1xvrXtOz5uHrtI4lrVkaElNBBLKznt4EiJegKKa3F9kD4yEOMfJ66NUT8X00\n1sqhBvBzV22zPn+cJYcafvhqp3NYh3ZT8VeeGUxlNzx5hQ0pO+0CQeyMaQslv3nK\n2CX5NdSHkN7gkEAWnh9TZdqcLNoTKgI8gghTwt1hZx39KC4W3BYGAR8fICSI1GIL\nHKzFPmS8cZUzxoIqgjnMm1rq2UTGU3X5W/Iuebiog+JcFaGPvBprcJqou5z7oYc+\nrlqYbx4vd1xeTRo9JQLY5OuWp7TYJitq6CSAaqtUTlDoF3XD5jFw4tl0HRIVbppf\nhhU0WMeyAKF0HGDTIzBQ+DBSZdfGhu1ETJypN82Bria5YuRBOZkESPfxHNBkWDNT\nYyLVsg+TIXh/KKhM9voCNAq21eOLl9jtFvPdEKWvsCuic+1uMoX3lz21PyRqQRJK\nyeY5Mo7ZHJzzrNsfvvB0dEEaZ04iGq8QHOTxA4rCH2Dr+S/1eBZQQvqe0zWAKBzU\nZolteiV5NIpixCSlSkZxqRgTRHwkiVnvR3/oKMWBhd1YFmXQNLf/WxxI2PiwUYBS\n7a6H1OgCt+WlWWJxY+St5tFJuleO2WjRMH+4ggLAo98mIkkmhBU8fSnlf3KI5sHN\nJEG/kZ79zGZFu9Y5abi9QylI8DcsDcaplcO17AFjkdtd6grNsEcZhsmMM0je83li\ne5O376XoFRv8+wnkelbK9k5TR+ouY38ZNEFOyzCnP5/l+p8k2RYNDO3nsVNXfuoE\naQUva1A4p7gZYjTyKpNi1l02Vd589OXvwgegiR+Lshs2D1++ZgWuhQG3BD81zoXy\nR7vqKXFuLpy64yXHv5d5q09/Xfb4zyBunlHP6cQyuxT/yL3RiTkl7/FoZpk/JD9x\nNheqhxM8bqEjX5LR6+//0ZKlkwBHsRVItAUjhhIbe3zjmFZyBmplyJjTZhk5FCmz\naP1AMocIYkzoAx7lR0F13wg4H3KqZoVZEToO/3dQlpYb3T+AUR22DxpUxq+h2s3g\n93Boa8jnbi/Hk+I/G1IPt31m2Nc58jNFQM9x37siEe9JI+A50yy3lfhK6EJb1qXS\nQUYGEdu+SE5AaTTG101NO86sdBmq3UHrCuklGEq06zFUecYO2FcCJonbOL6RFhT5\nkDinobHcJGg9SNUfOgwvSSep7QMPndoTH5LzLnwKNVBsXsdkyH7O0AEhbqctgYIu\nEbHYlmJCD2dahrDrpwwdU0CV3se7CARXOPblddGxCAQxsbH4Dj7FwQeDsrnafTyb\naWRVVpdqPuKR8TmTVGbTdoO8HUhdAtsH7qiuodx3zyl6UvdVJsohew1mw2Fv8/Cd\n"}

In [37]:
session_key = resultat["session_key"]
cle = decrypt_public(session_key,'key_private.pub')
print(cle)
payload = resultat["payload"]
s2file(payload, "payload.txt")

da3347fccc04bd86b34ab99803086f35


In [39]:
decrypter_fichier("payload.txt", cle)

Salut palmerjohn,

Tout d'abord, mettons les choses au clair. Je ne m'appelle pas anonymous.coward.

J'ai pu créer ce compte grace à mes privilèges d'administration, et, à ce que je
sache, il n'est pas lié à ma véritable identité. Je prends un certain nombre de
précautions pour que ça reste comme ça.

Tu devines ou tu sais déjà que tout ne va pas pour le mieux dans le meilleur des 
mondes. La police a un comportement aggressif envers tous ceux qui ne font 
qu'exprimer des désaccords ou même de simples doutes. Les canards boiteux sont 
accusés de "cyber-terrorisme" sous n'importe quel prétexte, et c'est 
suffisamment intimidant pour que tout le monde se la ferme. Ceux qui ne 
comprennent pas le premier avertissement ont des "accidents". Des gens 
disparaissent. Même les administrateurs de rang élevé, qui bénéficiaient 
du respect et de l'estime générale ne se sentent plus à l'abri maintenant.

Enfin, tu as du comprendre que la CLEA n'est qu'un des aspects de la situation,
et pas franche

In [8]:
def commande_stp(method, url, nom_fichier, K):
    requete = {'method': method, 'url': url}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/uVM/4919/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        for i in range(0, len(resultat)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(resultat[i])
            else : 
                f.write(resultat[i])

In [9]:
print(login)

palmerjohn


In [24]:
reponse = c.get("/bin/kerberos/authentication-service/palmerjohn")

#AUTHENTIFICATION
TGT = reponse['TGT']
Client_TGS_session_key = reponse['Client-TGS-session-key']
s2file(Client_TGS_session_key, "Client_TGS_session_key.txt")
Client_TGS_session_key = decrypt( "Client_TGS_session_key.txt", "j^5vL+Fpv@")

#AUTHENTIFICATEURS
d = {'username': 'palmerjohn', 'timestamp': time.time()}
e = json.dumps(d)
authentificateurs = encrypt(e, Client_TGS_session_key)

#OBTENTION D'UN TICKET DE SERVICE
dico = c.post("/bin/kerberos/ticket-granting-service", TGT= TGT, vm_name = 4919 , authenticator = authentificateurs)

#CONNECTION À LA MACHINE VIRTUELLE DÉSIRÉE
Client_Server_ticket = dico['Client-Server-ticket']
Client_Server_session_key = dico['Client-Server-session-key']
s2file(Client_Server_session_key, "Client_Server_session_key.txt")
Client_Server_session_key = decrypt("Client_Server_session_key.txt", Client_TGS_session_key)

d = {'username': 'palmerjohn', 'timestamp': time.time()}
e = json.dumps(d)
authenticator = encrypt(e, Client_Server_session_key)

reponse = c.post("/bin/uVM/4919/hello", ticket= Client_Server_ticket, authenticator= authenticator)
print(reponse)

Kerberos session established successfully. Access to /bin/uVM/4919 granted to palmerjohn.


In [27]:
reponse = commande_stp("GET", "/", "UVM0.txt", Client_Server_session_key )

In [12]:
reponse = c.get("/bin/kerberos/authentication-service/palmerjohn")

#AUTHENTIFICATION
TGT = reponse['TGT']
Client_TGS_session_key = reponse['Client-TGS-session-key']
s2file(Client_TGS_session_key, "Client_TGS_session_key.txt")
Client_TGS_session_key = decrypt( "Client_TGS_session_key.txt", "j^5vL+Fpv@")

#AUTHENTIFICATEURS
d = {'username': 'palmerjohn', 'timestamp': time.time()}
e = json.dumps(d)
authentificateurs = encrypt(e, Client_TGS_session_key)

#OBTENTION D'UN TICKET DE SERVICE
dico = c.post("/bin/kerberos/ticket-granting-service", TGT= TGT, vm_name = 0 , authenticator = authentificateurs)

#CONNECTION À LA MACHINE VIRTUELLE DÉSIRÉE
Client_Server_ticket = dico['Client-Server-ticket']
Client_Server_session_key = dico['Client-Server-session-key']
s2file(Client_Server_session_key, "Client_Server_session_key.txt")
Client_Server_session_key = decrypt("Client_Server_session_key.txt", Client_TGS_session_key)

d = {'username': 'palmerjohn', 'timestamp': time.time()}
e = json.dumps(d)
authenticator = encrypt(e, Client_Server_session_key)

reponse = c.post("/bin/uVM/0/hello", ticket= Client_Server_ticket, authenticator= authenticator)
print(reponse)

Kerberos session established successfully. Access to /bin/uVM/0 granted to palmerjohn.


In [13]:
reponse = commande_stp("GET", "/", "UVM0.txt", Client_Server_session_key )

NameError: name 'encrypt2' is not defined

In [44]:
decrypter_fichier("UVM0.txt", Client_Server_session_key)

HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL                                         HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL  LIGLUX v4.9.0-4          [u/VM guest]  HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL                                         HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL                                         HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL                                         HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL                                         HAL HAL HAL HAL HAL
HAL HAL HAL HAL HAL  L      I  GGGGG  L 

In [40]:
erreur =   b'Salted__\x8b\xa1<\x93\xb6\x00\xd2R\xef\x12(\xa2P\x9f\xf9\xb2\xea\xa1\x91\xa0R\xed\xd4\\:\r\xb2\xe4\x80c\xc2\xdaa%dt\xdbyX_L\xb3\x02\xc9=\x0f\xbb\xe8"\xe9\xe9\x14\xd9\xc6\x9a\xe4,\xde\x19\x92\xd6\xf2\xc7\x1a\xec\xd8\xf5\xa1W\xcd\xfe\xa8'

In [41]:
lecture_message_erreur(erreur, Client_Server_session_key )

uFS filesystem error: /bin/uVM/4919/chat-room not found.


In [43]:
nonce = c.post("/bin/login/stp", username= "michael60")

ServerError: ERREUR 400, Une fois qu'un tunnel chiffré est ouvert, les requêtes doivent être adressées à la passerelle (/uglix/bin/gateway).
Vous avez envoyé une requête à /uglix/bin/login/stp

In [ ]:
chapConnection('michael60vi')

In [78]:
K = K = "violates-"+nonce
requete = {'method': 'GET', 'url': '/bin/login/stp/handshake'}
requete_json = json.dumps(requete)
requete_chiffre = encrypt2(requete_json, K)
resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
resultat = base64.b64encode(resultat).decode()
with open("reponse.txt", 'w') as f: 
    for i in range(0, len(resultat)): 
        if (i % 64 == 0): 
            f.write("\n")
            f.write(resultat[i])
        else : 
            f.write(resultat[i])
reponse = decrypt("reponse.txt", K)

In [28]:
print(reponse)

STP login successful. Access to /home/michael60 granted.
[ You have new mail ]


In [32]:
def lecture_message_erreur(reponse,K): 
    reponse = base64.b64encode(reponse).decode()
    with open("reponse.txt", 'w') as f: 
        for i in range(0, len(reponse)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(reponse[i])
            else : 
                f.write(reponse[i])
    reponse = decrypt("reponse.txt", K)
    print(reponse)

In [39]:
def get_stp(url, nom_fichier): 
    requete = {'method': 'GET', 'url': url}
    requete_json = json.dumps(requete)
    requete_chiffre = encrypt2(requete_json, K)
    resultat = c.post_raw(url = '/bin/gateway', data = requete_chiffre, content_type='application/octet-stream')
    resultat = base64.b64encode(resultat).decode()
    with open(nom_fichier, 'w') as f: 
        for i in range(0, len(resultat)): 
            if (i % 64 == 0): 
                f.write("\n")
                f.write(resultat[i])
            else : 
                f.write(resultat[i])
    reponse = decrypter_fichier(nom_fichier, K)
    print(reponse)
    

In [42]:
get_stp('/bin/police_hq/ticket/1496', "piece_jointe")

Lors d'une opération routinière d'inspection des logs du système, un admin a
découvert que le programme /bin/fsck avait parfois un comportement surprenant.
Un extrait du log suspect se trouve en pièce-jointe (id="log").

Le programme /bin/fsck a manifestement été modifié de manière non-autorisée.
Ceci est très inquiétant car seuls des utilisateurs possédants un niveau de
privilège très élevé ont le droit de modifier des exécutables du système.

Tout ceci devient encore plus mystérieux et plus grave si on y ajoute le fait
que c'est michael60 qui a lancé /bin/fsck. Notre hypothèse de travail
actuelle est que michael60 cherchait à provoquer le déclenchement du code
malicieux, mais nous ne savons pas encore pourquoi.

Deux équipes vont travailler en parallèle : d'une part, l'équipe de retro-
conception se charge d'examiner le fichier /bin/fsck. Une étude préliminaire 
montre le code a été obfusqué avec des méthodes cryptographiques, donc il ne va
pas y avoir de résultats rapides. 

D'autre

In [35]:
get_stp('/bin/police_hq/ticket/1496/attachment/log', "piece_jointe")

fsck from UGLIX-util 2.20.1
ufsck 1.42 (29-Nov-18011)
/dev/flash0 contains a (clean) UglFS file system, manual check forced.
Pass 1 : Checking unodes, blocks, and sizes
Pass 2 : Checking (blocks vs files) maximum-flow
Pass 3 : Checking that block subgraph contains a (large) independent set
Pass 4 : Checking longest cycle in file subgraph
Pass 5 : Checking 3-colorability of dual file graph
Pass 6 : Checking for file deduplication issues
    [starting problem resolution routine]
    [MD5(/home/michael60/foo.bin) == MD5(/home/michael60/bar.bin)]
    [Files with identical hash detected: full check forced]
    [Comparing files...]
    ---> files are different
    ---> hash collision detected (probability 0.0000000000000000000000000001)
    ---> FSCK PANIC: UNRECOVERABLE ERROR. BYE BYE
        [EMERG*************************************************************** 
              *** Exception handler rerouted. P4wnV1rU$ starting          ***
              *** Oooh, colliding files. Interesting

In [95]:
get_stp('/bin/police_hq/ticket/1496/attachment/indication-0', "piece_jointe")

ServerError: ERREUR 403, b'Salted__kJ\xdc\xc3\x92\xa3\xb9\x94y\t\x9d\xb9I\xc1\xf1\xf1P\xe1o:U\xed\xd2\x19\x17\x81IMU\xb7\x0f\xaa\rr%|\x91\xd04xD1\x02YN\x1f\x0c\x8a\xc7\x8dQ\xb9R\xbf\xd9!'

In [38]:
get_stp('/bin/police_hq/ticket/1496/attachment/indication-1', "piece_jointe")

Le programme de recherche de collision fonctionne de la façon suivante : 

*) Il lit un "fichier préfixe" (éventuellement vide), dont la taille doit être
   un multiple de la taille des blocs.

*) Le préfixe est haché, ce qui aboutit à une valeur de l'état interne h_prefix:

    [IV] ---Prefix---> [h_prefix] 


*) A partir de là, un algorithme de recherche de collision est lancé. Cet
   algorithme produit deux séquences de deux blocs chacunes, (A_0, A_1) et
   (B_0, B_1), avec la propriété suivante :

   h_prefix ---A_0---> ??? ---A_1---> h_coll
   h_prefix ---B_0---> ??? ---B_1---> h_coll

Du coup, on a la collision :

    MD5(prefixe || A) == MD5(prefixe || B).

Mais notez que l'utilitaire n'inclut pas automatiquement le prefixe dans les
deux fichiers qu'il produit.


In [53]:
def recherche_collision(fichier_prefix): 
    with open("fichier_prefix.txt", "w") as f: 
        f.write("michael60")
        for i in range(0, 250-len("michael60")): 
            f.write("1")
    x = sha256(fichier_prefix.encode())
    x.digest()
    x.hexdigest()
print(recherche_collision("fichier_prefix.txt"))

None


In [39]:
get_stp('/bin/police_hq/ticket/1496/attachment/indication-2', "piece_jointe")

Pour mettre son nom d'utilisateur au début, il suffit de produire un préfixe (de
la bonne taille) qui commence par son nom d'utilisateur, et qui finisse par
n'importe quoi.


In [40]:
get_stp('/bin/police_hq/ticket/1496/attachment/indication-3', "piece_jointe")

Pour mettre "h4ckm0d3" à la fin, ce n'est pas beaucoup plus compliqué. L'astuce,
c'est qu'une fois qu'on a deux messages qui collisionnent, on peut les recycler
en rajoutant ce qu'on veut après.

En effet, on a :

    [IV] ---Prefix---> [h_prefix] ---A_0---> ??? ---A_1---> [h_coll]
    [IV] ---Prefix---> [h_prefix] ---B_0---> ??? ---B_1---> [h_coll]

Comme, de façon bien commode, les chaines : (Prefix||A) et (Prefix||B) ont une
taille qui est un multiple de la taille du bloc, tout ce qu'on pourrait rajouter
après va aller dans un nouveau bloc, et ne va donc pas perturber la collision :

    [IV] ---Prefix---> [h_prefix] ---A---> [h_coll] ---Suffix---> [h_suffix]
    [IV] ---Prefix---> [h_prefix] ---B---> [h_coll] ---Suffix---> [h_suffix]

Notez que le mécanisme de paddign va entrer en action dans le suffixe, mais que
ce n'est pas un problème.


In [41]:
get_stp('/bin/police_hq/ticket/1496/attachment/indication-4', "piece_jointe")

La partie un peu plus délicate consiste à produire non pas DEUX fichiers qui
collisionnent, mais QUATRE. Pour cela, il faut utiliser la technique suivante,
inventée par Antoine Joux en 2004.

Partant d'une valeur de l'état interne quelconque (disons h_0), on trouve une
première collision :

    [h_0] ---A---> [h_1]
    [h_0] ---B---> [h_1]

Puis, partant de h_1, on trouve une deuxième collision :

    [h_1] ---C---> [h_2]
    [h_1] ---D---> [h_2]

On a donc :

    [h_0] ---A---> [h_1] ---C---> [h_2]
    [h_0] ---A---> [h_1] ---D---> [h_2]
    [h_0] ---B---> [h_1] ---C---> [h_2]
    [h_0] ---B---> [h_1] ---D---> [h_2]

Partant de l'état interne h_0, on a obtenu 4 séquences de 4 blocs qui
aboutissent au même état interne h_2 : AC, AD, BC, BD.

Concrètement, h_0 est l'état interne obtenu après avoir traité le préfixe.
Pour que l'utilitaire de recherche de collision cherche une collision à partir
de h_1, il faut lui donner un nouveau préfixe qui aboutisse à h_1. On peut lui
donner, par exe

In [32]:
get_stp('/bin/fsck', 'fsck.txt')

fsck from UGLIX-util 2.20.1
ufsck 1.42 (29-Nov-18011)
/dev/flash0 contains a (clean) UglFS file system, manual check forced.
Pass 1 : Checking unodes, blocks, and sizes
Pass 2 : Checking (blocks vs files) maximum-flow
Pass 3 : Checking that block subgraph contains a (large) independent set
Pass 4 : Checking longest cycle in file subgraph
Pass 5 : Checking 3-colorability of dual file graph
Pass 6 : Checking for file deduplication issues
fsck completed successfully.


In [67]:
get_stp('/home/michael60', 'foo.txt')

# Bienvenue, michael60.
# Dernière connection le 13-02-21893 16:33:41 (protocole : STP)
# Vous avez 2 messages non lus (4 en tout).

Rumeur du jour: 

Il y aurait des missions non-documentées.

***** Contenu de votre dossier personnel ******

  /home/michael60/TODO
  /home/michael60/md5_collider.tar.gz
  /home/michael60/research.jpg
  /home/michael60/chat.log



In [96]:
get_stp('/home/michael60/research.jpg', 'foo.txt')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [66]:
get_stp('/home/michael60/TODO', 'foo.txt')

A) FSCK

injecter collision
vérifier système de fichier (/bin/fsck)
---> bug
suivre directives

Collision ?

* MD5(A) == MD5(B) == MD5(C) == MD5(D) and len({A,B,C,D}) == 4

 ^
/!\     Tous les fichiers commencent par le nom d'utilisateur UGLIX
---     Tous les fichiers finissent par 'h4ck m0d3'
 |

Solution la plus simple?


B) STOCKAGE PÉRENNE

C'est quoi le problème avec la bande magnétique numéro 4919 ? Pourquoi l'opérateur
sort une excuse bidon ?


In [46]:
reponse = b'Salted__\xbc\x98\xec\xdb\xb4(z0\xec\x88~\xb8U\x97nh]D\xe9sL]\xcd\xa7'
lecture_message_erreur(reponse)

File not found.


In [33]:
get_stp('/bin/police_hq/ticket/1496', 'ticket_1496.txt')

Lors d'une opération routinière d'inspection des logs du système, un admin a
découvert que le programme /bin/fsck avait parfois un comportement surprenant.
Un extrait du log suspect se trouve en pièce-jointe (id="log").

Le programme /bin/fsck a manifestement été modifié de manière non-autorisée.
Ceci est très inquiétant car seuls des utilisateurs possédants un niveau de
privilège très élevé ont le droit de modifier des exécutables du système.

Tout ceci devient encore plus mystérieux et plus grave si on y ajoute le fait
que c'est michael60 qui a lancé /bin/fsck. Notre hypothèse de travail
actuelle est que michael60 cherchait à provoquer le déclenchement du code
malicieux, mais nous ne savons pas encore pourquoi.

Deux équipes vont travailler en parallèle : d'une part, l'équipe de retro-
conception se charge d'examiner le fichier /bin/fsck. Une étude préliminaire 
montre le code a été obfusqué avec des méthodes cryptographiques, donc il ne va
pas y avoir de résultats rapides. 

D'autre

In [52]:
mail("palmerjohn", '13236')

----------------------------------------------------------------------
Message #13236.


Date: 12-02-21893 22:47:35
From: UGLIX Management
Subject: [work] Invitation

Chèr(e) palmerjohn,

Je ne peux que constater qu'on ne nous avait pas menti en nous vantant vos
capacité. Vous les démontrez en fermant ticket sur ticket. 

Puisque vous avez manifestement des capacité, le management a décidé de 
vous conseiller de participer au programme de formation le plus exigeant : la 
UGLIX Crypto Hackademy. Je viens d'apprendre que votre candidature vient d'être
examinée, et retenue. Toutes mes félicitations.

La UGLIX Crypto Hackademy délivre des certifications très recherchées, qui vous
permettront de prétendre à des positions plus intéressantes, tant 
intellectuellement que financièrement.

Je vous prie donc de partager votre temps entre vos obligations professionnelles 
à la CLEA d'une part, et votre formation sur /bin/hackademy d'autre part.

Avec tous mes voeux de réussite,
Bien cordialement,

In [9]:
reponse = c.get("/bin/hackademy")

In [10]:
print(reponse)

[initializing ticket system]         o
[connecting to tickets daemon]   .-""|
[fetching tickets]               |-""|
---> 4  tickets                      |   ._--+.
[building UI]                       .|-""      '.
[building training plan]          +:'           '.
[startup complete]                 | '.        _.-'|
                                   |   +    .-"   J
                _.+        .....'.'|    '.-"      |
           _.-""   '.   ..'88888888|     +       J''..
        +:"          '.'88888888888;-+.  |    _+.|8888:
        | \         _.-+888888888_." _.F F +:'   '.8888'....
         L \   _.-""   |8888_.-"  _." J J J  '.    +88888888:
         |  '+"        |_.-"  _.-"    | | |    +    '.888888'._''.
       .'8L  L         J  _.-"        | | |     '.    '.88_.-"    '. 
      :888|  |         J-"            F F F       '.  _.-"          '.
     :88888L  L     _+  L            J J J          '|.               '; 
    :888888J  |  +-"  \ L          _.+.|.+.          F '.    

In [119]:
print(c.get('/bin/hackademy/ticket/1500'))

Vous devez résoudre l'équation : 

    a * X + b  ==  0    modulo n

Les valeurs de a, b et n sont dans des pièces-jointes du même nom. Quand vous
aurez déterminé la valeur de X, envoyez une requête à l'adresse : 

    /bin/hackademy/exam/arith/eq-lin-mod-p

avec un paramètre "X".
[ You have new mail ]


In [27]:
def euclide_etendu(a, b):
    a = int(a)
    b = int(b)
    r = [a, b]
    u = [1, 0]
    v = [0, 1]
    i = 1
    
    while(r[i] != 0): 
        q = r[i-1]//r[i]
        r.append(r[i-1]-q*r[i])
        u.append(u[i-1]-q*u[i])
        v.append(v[i-1]-q*v[i])
        i = i+1
    return (r[i-1], u[i-1], v[i-1])

In [211]:
r, u, v = euclide_etendu(a, n)

In [212]:
print("r = {}, u = {}, v = {}".format(r, u, v))

r = 1, u = 21751254074263731491111262357858438695947547955422857490579103624162044703353967602945250191566851365538284966952231557601346687636637200384606278722578694493969189594511885852469467359766517993541250189299210686710109007467053457259038736212723539209289223372431948207644572221213255343713660586395568471095923742101467596108636207303417709493469212879277505088240366755416982974974386618766098140112822870574705951961081100003950597193200955934732094887376499251650285133028696140632346659303927127956929148823130477908754224166382781401333749887805329802067517163517698595009552414493206668760811258768917724747843132552805685021371141432534329842125372738163742258007909961485354448761680767412117090031736241595454616880766797133958966860531693049172305688118736510315990974186671127134595932751888293033717635326740691381595491463177821698229415399321078422994456700105478051758970856758275836976148815705538670951701094581846793900329195146209257566726604666292720610762483303475589

In [210]:
a = c.get('/bin/hackademy/ticket/1500/attachment/a')
b = c.get('/bin/hackademy/ticket/1500/attachment/b')
n = c.get('/bin/hackademy/ticket/1500/attachment/n')
print("a = {}\n , b = {}\n, n = {}".format(a, b, n))

a = 549263592333587317906701844885240892350756539204276719118028166998555369525708608263967458593924555434668034075122699534583625864135015760791087157598512353170406565760301784067877921710310804011211667395879318186365110098036633782079341298411260779152631017081448334621743435593601869053076641703563953262790285143973639673666821471565830841137809962447483523480241416054518512883084518458674647395877916273586430457508089270978022628005367637728006558638802250809734823056142966635206274055783583262800329230273101874202604975130690282916946163930264627333364328958712532680445716834808722759126376538242275081988260209586218145720846421593634690147481905574400322229917346953592828856228811286964764652399080336748211910366626359120241428491802699443692139626854420892260452634576012693124907812983941658816595685027191152134964593495790075981878411508153026555709144946936116958703672638992484221318725826444570843970201384421230975239204726524518277795811057128179470622867240703870844489356

In [213]:
b= int(b)
u = int(u)
X = -b*u
c.post("/bin/hackademy/exam/arith/eq-lin-mod-p", X=X)

'LINEAR.EQUATION:3030:2@459700|6742a187fabf53a5bda7a6791d690a2a\n[ You have new mail ]'

In [33]:
c.post('/bin/hackademy/ticket/1500/close')

'STATUS : OK --- ticket 1500 closed\n[ You have new mail ]'

In [70]:
print(c.get('/bin/hackademy/ticket/1501'))

Un nombre n est fourni dans une pièce-jointe du même nom. Ce n'est pas un nombre
premier. Vous devez fournir une PREUVE qu'il est composite. Déterminez un
témoin de Fermat, c'est-à-dire un entier a tel que :

    a^(n-1) != 1   modulo n 

Quand vous l'aurez trouvé, envoyez-le à :
   
	/bin/hackademy/exam/prime/fermat-disproof

dans un paramètre a.

Remarquez que vous n'avez pas besoin de le factoriser.
[ You have new mail ]


In [191]:
n = c.get('/bin/hackademy/ticket/1500/attachment/n')
print("n = {}".format(n))

n = 102087548682999670530649673977126443980885461686071717689601373164505624532727032975207190739276284356169603107536471247651611145055400032629628889778334661578438146337955081148378589307253448268773897486789760345216233432269930699543559508128433153231951895342740805234998887715383377142877552561108802679244257807130061797312175085821219273377859809951743072566298755270458396269575203548087462402350743481113886532698514719461423179054079176906318308983353939481930366587360591741954978508587679496845170076281206416296864153565978559447185453328896965395573516153590385565254377485853923617509449597395445200533185031208427850613193721780019675409022082443031321775753616632396108553854221821340818019180703477865408473551378969071745072706422139346367855733366834486039267473628969790358250979204035567620121496614774940299855166734579282560397719322666173365029135446562229454798391645415389455415374005418473936818492641506200453351972124313555695326638228860867892126717316351154006793423

In [192]:
a = 2*n
c.post("/bin/hackademy/exam/prime/fermat-disproof", a = a )

'PRIME.OU.PAS:3030:1@135953|b6c1abc089a99bae7cbf2a2a74a13b0a\n[ You have new mail ]'

In [34]:
c.post('/bin/hackademy/ticket/1501/close')

'STATUS : OK --- ticket 1501 closed\n[ You have new mail ]'

In [13]:
print(c.get('/bin/hackademy/ticket/1502'))

Vous devez déterminer un nombre premier appartenant à un intervalle [a; b[ qui
vous est imposé (a et b sont en pièce-jointe).

Quand vous l'aurez trouvé, envoyez-le à :
   
	/bin/hackademy/exam/prime/range

dans un paramètre p.
-----
Il y a une indication (pj = 'indication-0').
[ You have new mail ]


In [14]:
a = c.get('/bin/hackademy/ticket/1502/attachment/a')
b = c.get('/bin/hackademy/ticket/1502/attachment/b')
print("a = {}".format(a))
print("b = {}".format(b))

a = 166620083278534922569961260699721842837997172039698165772696312984585091397837831448310027909370501148424458867043459270497522466337192658441805943958555941580056920722759897405333750447829404083895368445011019426421431177334269019893314181067989225232303814060279763370129354800763686284271132814018107635783
b = 166620083278534922569961260699721842837997172039698165772696312984585091397837831448310027909370501148424458867043459270497522466337192658441805943958555941580056920722759897405333750447829404083895368445011019426421431177334269019893314181067989225232303814060279763370129354800763686284271132814018107649095


In [15]:
indication = c.get('/bin/hackademy/ticket/1502/attachment/indication-0')
print(indication)

On peut gagner un peu de temps utilisant le fait qu'un nombre pair n'est pas
premier (sauf si c'est 2). Technique un peu améliorée de production de nombre 
premier :

i = <nombre aléatoire IMPAIR de la bonne taille>
while True:
    <test de primalité sur i>
    i += 2



In [16]:
import sympy
def trouver_nbr_premier(a, b): 
    n = a
    resultat = 0
    while(n < b): 
        if (n%2 != 0): 
            if (sympy.isprime(n) == True): 
                resultat = n
        n = n+1
    return resultat 

In [17]:
a = int(a)
b = int(b)
resultat = trouver_nbr_premier(a, b)
print(resultat)

166620083278534922569961260699721842837997172039698165772696312984585091397837831448310027909370501148424458867043459270497522466337192658441805943958555941580056920722759897405333750447829404083895368445011019426421431177334269019893314181067989225232303814060279763370129354800763686284271132814018107648551


In [233]:
c.post("/bin/hackademy/exam/prime/range", p= resultat )

'PRIME.RANGE:3030:2@680158|b9f69779b286eabd6cd0d39604d7350e\n[ You have new mail ]'

In [35]:
c.post('/bin/hackademy/ticket/1502/close')

'STATUS : OK --- ticket 1502 closed\n[ You have new mail ]'

In [152]:
print(c.get('/bin/hackademy/ticket/1503'))

Vous devez déterminer des nombres premiers p_0, p_1, ..., p_k de telle sorte que
leur produit appartienne à l'intervalle [a; b[ qui vous est imposé (a et b sont
des pièces jointes).

La taille des p_i doit être inférieure à 64 bits. De plus, tous les p_k doivent
être différents. 

Quand vous les aurez trouvés, envoyez-les à :
   
	/bin/hackademy/exam/prime/product

dans un paramètre p. Il doit s'agir d'une liste d'entiers.
-----
Il y a des indications (pj = 'indication-[01]').
[ You have new mail ]


In [153]:
a = c.get('/bin/hackademy/ticket/1503/attachment/a')
b = c.get('/bin/hackademy/ticket/1503/attachment/b')
#print("a = {}".format(a))
#print("b = {}".format(b))

In [249]:
print(len(str(b)))

616


In [236]:
indication0 = c.get('/bin/hackademy/ticket/1503/attachment/indication-0')
indication1 = c.get('/bin/hackademy/ticket/1503/attachment/indication-1')

In [238]:
print("indication 0 :\n{}".format(indication0))

indication 0 :
Pour produire k nombres premiers p_1, ..., p_k de telle sorte que 
    
    Q = p_1 * p_2 * ... * p_k 

appartienne à l'intervalle [a; b[, il n'y a pas 36 solutions. En gros, la
technique est la suivante : 

   1) Fabriquer dans un premier temps les k-1 premiers nombres librement
   2) Fabriquer le dernier en faisant bien attention à ce que produit soit
      dans [a; b[

Après la phase 1, on se trouve avec Q' = p_1 * p_2 * ... * p_{k-1}. Pour que le
produit final soit dans le bon intervalle, il faut choisir p_k dans l'intervalle
[a/Q'; b/Q'[.


In [239]:
print("indication 1 :\n{}".format(indication1))

indication 1 :
Pour mettre en oeuvre la stratégie précédente, il y a deux problèmes à éviter :

   *) Si Q' est trop petit, alors l'intervalle [a/Q'; b/Q'[ ne contient que des
      nombres trop grand, et on ne pourra pas choisir dedans des nombres de 64
      bits ou moins.

    ---> Il faut que |b/Q'| <= 64, donc que Q' >= b / (2**64).
   
   *) Plus Q' est grand, plus l'intervalle [a/Q'; b/Q'[ rétrecit. Et s'il est
   	  trop petit, il peut ne pas contenir de nombre premier. Il est difficile
   	  d'obtenir des certitudes, mais il est raisonnable de penser qu'il faut
   	  qu'il contienne O(|b|) nombres différents pour que l'un d'entre eux soit
   	  premier avec bonne probabilité.

    ---> Il faut que (b/Q') - (a/Q') >= [constante arbitraire] * |b|

Pour s'en sortir, il faut donc :

*) déterminer dans un premier temps la taille que doit avoir Q' en bits.
*) produire Q' en multipliant ensemble des nombres premiers arbitraires.
*) trouver le dernier facteur premier.


In [143]:
liste_p = [5]
p = 5
for i in range(0,224): 
    p = sympy.nextprime(p)
    liste_p.append(p)
#print(liste_p)

Q_prim = 1
for i in liste_p: 
    Q_prim = Q_prim * i

print("test {}".format(Q_prim > (b//(2**64))))
print("test {}".format(Q_prim < ((b-a)//616)))

pk = sympy.nextprime(a//Q_prim)
liste_p.append(pk)

Q = 1
for i in liste_p: 
    Q = Q * i
    
print(Q>a)
print(Q<b)

test True
test True
True
True


In [144]:
resultat = c.post("/bin/hackademy/exam/prime/product", p =liste_p)

In [145]:
print(resultat)

PRIME.PRODUCT:3030:4@277389|bd1f59ada85685867c0f537f58b5a0a2
[ You have new mail ]


In [36]:
c.post('/bin/hackademy/ticket/1503/close')

'STATUS : OK --- ticket 1503 closed\n[ You have new mail ]'

In [10]:
print(c.get('/bin/hackademy/ticket/1504'))

Vous devez déterminer un nombre premier sûr appartenant à un intervalle [a; b[
qui vous est imposé (a et b sont en pièce-jointe). Pour rappel, un nombre
premier sûr est un nombre de la forme 2p + 1, où p est lui-même premier.

Quand vous l'aurez trouvé, envoyez-le à :
   
	/bin/hackademy/exam/prime/safe

dans un paramètre p.
-----
Il y a deux indication (pj = 'indication-[01]').
[ You have new mail ]


In [11]:
a = c.get('/bin/hackademy/ticket/1504/attachment/a')
b = c.get('/bin/hackademy/ticket/1504/attachment/b')
#print("a = {}".format(a))
#print("b = {}".format(b))

In [12]:
indication0 = c.get('/bin/hackademy/ticket/1504/attachment/indication-0')
indication1 = c.get('/bin/hackademy/ticket/1504/attachment/indication-1')

In [15]:
print(indication0)

Pour fabriquer un nombre premier sûr, la technique est simple : générer un 
nombre aléatoire q, vérifier s'il est premier, puis vérifier que 2q+1 est 
premier. 

Si q est un entier choisi au hasard, ceci va arriver une probabilité qu'on peut 
estimer à O(1/n^2), où n désigne la taille souhaitée en bits. On peut donc 
s'attendre à devoir faire O(n^2) essais.

En pratique, ça peut prendre plusieurs heures.


In [28]:
print(indication1)

Une technique permet cependant de gagner du temps : 

si q est premier, alors il est impair, et on a : q == 2*q' + 1. 

Alors p s'écrit :

    p == 2q+1 == 4q' + 3.

Or, on voit maintenant que p est un multiple de 3 dès lors que q' est un 
multiple de 3. Il ne faut donc pas que q' soit un multiple de 3.

Les valeurs possibles de q' sont donc :

q' == 3q'' + 1         et  q' == 3q'' + 2

Les valeurs de q correspondantes sont donc :

q == 6q'' + 3          et  q  == 6q'' + 5

Et donc q ne pourra pas être premier dès que q'' est un multiple de 3 ou de 5.

Les valeurs de p correspondantes dont donc :

p == 4(3q'' + 1) + 3   et  p == 4(3q'' + 2) + 3
  == 12q'' + 7             p == 12q'' + 11

Et donc, p ne peut pas être premier si q'' est un multiple de 7 ou de 11. On 
peut donc se contenter de faire :

    1. Choisir q'' non multiple de 3, 5 7 et 11.
    2. q <--- 6q'' + 3
    3. Si q n'est pas premier, aller en 6.
    4. p <--- 12q'' + 7
    5. Si p n'est pas premier, aller en 6. Sinon c'

In [22]:
from random import *
def trouver_nbr_premier_sur(a, b): 
    while True: 
        q = randint(a//12, b//12)
        while ((q%3 == 0) | (q % 5 == 0) | (q %7 == 0) | (q %11 == 0)): 
            q = randint(a//12, b//12)
        q = 6*q+3
        if(sympy.isprime(q) == True): 
            p = 2*q+1
            if (sympy.isprime(p) == True): 
                return p
        q = q + 2
        if(sympy.isprime(q) == True): 
            p = 2*q+1
            if(sympy.isprime(p) == True):
                return p

In [23]:
a = int(a)
b = int(b)
#print(a)
#print(b)
resultat = trouver_nbr_premier_sur(a, b)

In [24]:
print(resultat)

29525598985541801357602479587090042292002781963403240648009860415484329602461632749919494715608149617637787329254556679211777085500896449163332732379125747079700077592024762083724079059627874616758075472809302239542247058730910152716240188970978826840388777106784276747145090151398147955355467171754631679601523205804435096846407462990521940026243772612160637375126605998473792504724468460164270106657675442570158156291025891832246374124684040092288703915478540784020496625656999703384951029586144136735636500750014403707008151799128044927821394300592249555002618799393502065056986039862012839285066413412987446688619


In [25]:
print(resultat > a)
print(resultat < b)
print(sympy.isprime(resultat))

True
True
True


In [26]:
c.post("/bin/hackademy/exam/prime/safe", p = resultat)

'PRIME.SAFE:3030:3@622980|793aaf19684d8908973abc4233ddca07\n[ You have new mail ]'

In [38]:
c.post('/bin/hackademy/ticket/1504/close')

'STATUS : OK --- ticket 1504 closed\n[ You have new mail ]'

In [11]:
print(c.get('/bin/hackademy/ticket/1505'))

Vous devez produire un nombre premier p appartenant à l'intervalle [a; b[ qui
vous est imposé (a et b sont en pièce-jointe), ainsi qu'une racine primitive 
modulo p. Pour que le jury puisse vérifier que vous avez réussi, vous devrez 
également fournir la factorisation de p-1.

Pour des questions de sécurité, p-1 doit avoir au moins un facteur premier 
de 384 bits ou plus.

Quand vous aurez réussi, connectez-vous à :

    	/bin/hackademy/exam/discrete-log/primitive-root

Vous devez envoyer deux paramètres :
  + g     : une racine primitive modulo p
  + p_m_1 : la liste des facteurs premiers de p-1
---
Il y a des indications (pj = 'indication-[012]').
[ You have new mail ]


In [12]:
a = c.get('/bin/hackademy/ticket/1505/attachment/a')
b = c.get('/bin/hackademy/ticket/1505/attachment/b')

In [13]:
indication0 = c.get('/bin/hackademy/ticket/1505/attachment/indication-0')
indication1 = c.get('/bin/hackademy/ticket/1505/attachment/indication-1')
indication2 = c.get('/bin/hackademy/ticket/1505/attachment/indication-2')

In [14]:
print(indication0)

Le problème principal consiste à produire un nombre premier p tout en
connaissant la factorisation de p-1. La seule procédure envisageable consiste à
produire des nombres X de la bonne taille dont la factorisation est connue, puis
à tester si X+1 est bien premier.


In [15]:
print(indication1)

La technique la plus simple, mais pas la plus rapide, consiste à générer un
nombre premier sûr. Si p = 2q + 1, alors la factorisation de p-1 est connue, 
c'est 2*q.


In [16]:
print(indication2)

Il est possible de faire plus rapide. La rumeur prétend que certains h4x0rz ont
écrit du code Python pur qui résoud le problème en 5 secondes environ.

Pour aller plus vite, il faut éviter au maximum de faire des tests de primalité
sur des grands entiers. Pour cela, il faut fabriquer p-1 avec plusieurs petits
facteurs premiers, puis tester si p est premier.

C'est grosso-modo la même chose que lors de calcul d'un produit de facteurs
premier dans un intervalle donné. La différence, c'est qu'il faut soigneusement
déterminer l'intervalle dans lequel on cherche le DERNIER facteur : il faut 
qu'il soit suffisament grand pour qu'on puisse trouver dedans non pas UN, mais 
DES nombres premiers. En effet, on fabrique p-1 aléatoirement, mais p n'est pas 
automatiquement premier.


In [20]:
a = int(a)
b = int(b)
def trouver_nbr_premier_sur_2(a, b): 
    while True: 
        q = randint(a//12, b//12)
        while ((q%3 == 0) | (q % 5 == 0) | (q %7 == 0) | (q %11 == 0)): 
            q = randint(a//12, b//12)
        q = 6*q+3
        if(sympy.isprime(q) == True): 
            p = 2*q+1
            if (sympy.isprime(p) == True): 
                return p, q
        q = q + 2
        if(sympy.isprime(q) == True): 
            p = 2*q+1
            if(sympy.isprime(p) == True):
                return p, q
            
p, q = trouver_nbr_premier_sur_2(a, b)
print("p = {}".format(p))
print("q = {}".format(q))

p = 2632598649965098970217059027840872691332451317020873875143867092777537849716975464001339804122245252691805487167770107424274085481435460738110778720544529676425137988574366066668276127882375437468139675476464498893806984824170770493570388055161469067833103037263821155810564803566771459113304809708642109789134709548230204223486205760310233319534426990925146650282150478011091422399646504069732877894125281438111447389583437454693110661344459624528797939160116064899351969973026114016967020900831020020191321491873149066596767621598066842705337489717914303492036656563387636803694735106309309856815054576689339773047
q = 131629932498254948510852951392043634566622565851043693757193354638876892485848773200066990206112262634590274358388505371213704274071773036905538936027226483821256899428718303333413806394118771873406983773823224944690349241208538524678519402758073453391655151863191057790528240178338572955665240485432105489456735477411510211174310288015511665976721349546257332514107523900554

In [21]:
print(a<p)
print(p<b)

True
True


In [22]:
p_m_1 = 2*q

In [148]:
def liste_nbr_premier(a, b): 
    p = 2
    liste = [p]
    while (p < a): 
        p = sympy.nextprime(p)
        liste.append(p)
    return liste 

In [149]:
a = int(a)
b = int(b)
liste = liste_nbr_premier(a, b)
print(liste)

KeyboardInterrupt: 

In [13]:
print(c.get('/bin/hackademy/ticket/1511'))

Vous devez générer une paire de clefs RSA. Pour plus de piquant, l'exposant
public ("e") vous est imposé. Vous le trouverez en pièce-jointe (id="e").

Vous devez envoyer la partie ``modulus'' de la clef publique (le "n") à
l'adresse :

    /bin/hackademy/exam/rsa/keygen

Vous récupérez en échange un message chiffré avec la clef publique RSA que vous
venez de produire. Vous devez le déchiffrer. Le clair représente une chaine de 
caractères (cf /doc/ints, "CONVERSION int() <---> bytes()").

[ You have new mail ]


In [14]:
e = indication0 = c.get('/bin/hackademy/ticket/1511/attachment/e')
print(e)


262866018186790498373512827038848736241


In [15]:
def gen_key_private_modif_expo(e):
    args = ['openssl', 'genpkey', '-algorithm', 'RSA', '-pkeyopt', 'rsa_keygen_bits:2048', '-pkeyopt', 'rsa_keygen_pubexp:'+e]
    result = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    with open('key_private_modif_ex.pub', 'w') as f:
        f.write(result.stdout.decode())
        
def trouver_key_public(file_key_private): 
    args = ['openssl', 'pkey', '-in', file_key_private, '-pubout']
    result = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode()
    
def visualiser_key_RSA(maCle): 
    #openssl rsa -in maCle.pem -text -noout
    args = ['openssl', 'rsa', '-in', maCle, '-modulus', '-noout']
    result = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.stdout.decode()
    

In [16]:
gen_key_private_modif_expo(e)

In [17]:
key_public_RSA = trouver_key_public('key_private_modif_ex.pub')
print(key_public_RSA)

-----BEGIN PUBLIC KEY-----
MIIBMDANBgkqhkiG9w0BAQEFAAOCAR0AMIIBGAKCAQEA3sYwZGV4wmpG61ctmvVy
HFBiLBSPqz2fFQhMMOm73PD3PorcG8J/tapVPA4EeOB5gmDiA0JxtSks94OCSe3k
xRBa9WVhABDuvAYqbzKL3l9W1PyGJDT+GQ2t2twrWdqtM3b2axqxlDUvDtOyRztO
qDN6yeLCZ7R6CMez2u7aG43FLGQ+kVqqJVNfHzSDxJYXO6WK9A/1g0n8MTf6icBD
tKSq1S7dQyd4XiN9n8vMTrtQyPAeTVNmxZ6jn6vihdur3jNfyxM7WnhXUGOGrt9h
aiGATLZVwZ/c/blOVbnvy/p8k4Uoo2Qa32s+aYev2asVCS0/Z5vucyuwBMQ8nbON
fwIRAMXCJ1DwVK13QTk6Y2e/W/E=
-----END PUBLIC KEY-----



In [18]:
def sup_deux_point(nbr): 
    resultat = ''
    for i in nbr: 
        if i != ':': 
            resultat += i
    return resultat

In [19]:
contenut = visualiser_key_RSA('key_private_modif_ex.pub')
modulus = contenut.split('=')[1]
modulus = int(modulus,base=16)
#print(contenut)
#modulus = "00:d0:8c:ce:a7:be:8f:01:2b:e9:d1:25:46:61:87:0a:48:e1:79:2c:0c:5a:59:5e:37:6a:d6:32:8a:22:7e:01:f0:c7:11:ce:c3:aa:27:e1:5c:d2:30:ef:0c:96:32:91:71:c3:10:e9:74:6e:57:fa:93:ef:1e:bb:be:95:7c:a6:9a:fe:90:41:f7:a0:53:ae:e7:4e:d8:66:a6:ab:6d:bb:f9:6f:52:79:f6:f8:7e:20:65:bf:0f:6e:e7:ab:d8:0b:7b:85:47:c4:e2:13:bd:a3:1e:0e:82:64:b9:83:24:e2:bb:23:f1:ca:c1:c0:f1:5c:e4:eb:ad:4e:aa:f6:6f:0f:42:cf:7c:82:c9:3a:5c:bf:7a:80:38:49:96:48:2a:07:93:f1:db:32:af:b6:b9:8e:90:1e:a1:09:db:f9:41:4c:12:ce:3b:8c:5a:ba:bd:8c:43:08:00:9e:83:36:56:9b:54:f9:4b:ac:ad:f4:3d:41:70:ea:5a:33:8b:20:43:60:2b:99:3d:f1:eb:63:18:4c:77:7d:aa:9c:9b:65:f9:04:e6:32:08:1c:86:19:43:65:16:84:80:2f:5c:e5:64:6d:a0:74:4a:08:b9:c2:66:1b:6f:f7:c2:15:0d:4c:54:74:d7:e4:ed:2e:07:3a:7e:10:c7:cc:d9:ca:d8:27:92:52:75"
#modulus = sup_deux_point(modulus)
#print(modulus)
#modulus = int(modulus, base = 16)
print(modulus)

28122634356579624113663612825355303529237857817151812491317775711902183036635070504391797979096244156830920389139818598740042430869904348326872846382329769153361452960567828351308663637199010918618532528934764325803320728339004808449025088382264317575678300431881722845272079138630145261866804237474699176175363854010787319780159444074910090762258454247837651564978906656852312485561114372428626473597135561573606258843820447059840366401347225522909556480290612558231066152843093030435745084490763547043239135764940137908891878535784859188714338495345582954158679569437956764537128789390442510558450292797087390338431


In [20]:
reponse = c.post('/bin/hackademy/exam/rsa/keygen', n= modulus)

In [21]:
print(reponse)

{'ciphertext': 21062655549565302670013359446153779044641070751642809565165464254958934900859164944092836495463436942121975098591594967026103571168568555911597856673536859007048618338746536222921362727613403087120561939823109316120758776350937811564355692369388227115988910357160125151449094181989632036657286283431779734621065719176424330981387220379677948544044862359968217766958371644834462480626167621170230014691859537778642191104730118890259632514097704366931609126212860384259080668693601742130859834206175586871826611853831439566972560622300894679814983109456403986810446058385897822802567286040846196365834474654568184006081}


In [43]:
#x = reponse['ciphertext']
#x = x.to_bytes(length=256, byteorder="big")
#x
x = reponse['ciphertext']
x = hex(x)
print(x)

0xa6d92ed89a15e862039beda148cf1bdf0afd80510a37bffb9e30991b726ef1ddf1a216f389adf1f7b0119be1ff1446a44bc6697a7618ffbfc715e5d1a467d1ab09c1c68eb6ded88068e9fee34575422dc5627f117b76a601ad01521a25ffadd5ae09b3476531e73b873332580db2c6503e06adfb22c2536a17a67a3da45b97a59ea4d7f642f522fbcedc68880734225cbbe44911bd65aa1e3669f8575185cd28733ba666bed536baff0e280eb92a85d43d9c77185d66b2d5bf6f1652565e9f7b10ef4963d0fd4ed51cf39c6c6ed95ae973156ae7de42e94febd9e7a581fc50ff6eb29992a7e4c7a8a779fc3ac629082728d2de4c1abb654b4597daa25187c5c1


In [48]:
ciphertext = str(reponse['ciphertext'])
i = int(ciphertext, base=16)
h = "{}".format(i)
h = base64.b16encode(h)
print(h)

TypeError: a bytes-like object is required, not 'str'

In [253]:
ciphertext = (reponse['ciphertext'])
ciphertext = hex(ciphertext)[2:]
ciphertext = base64.b16decode(ciphertext, casefold=True)
#print(ciphertext)
ciphertext = base64.b64encode(ciphertext).decode()
print(ciphertext)

Up1FwQJSMli5tGDg1kSGqCrFQd9p/cldyA4BEhSpNkI0F8940cCSzWR74nxbvUHr6TNo35EPTZcwEV1JnuWSp4oRO2fe1tOYRrQdLApBZDaC3GgEt3ws6iGj5YzgNUtdmDg4kG9SeQesaZXcTCv1mUe7BDa8ZNCUICnIDMX2lawBTBpx6OB+IXoVWfndh9X5vQj2oroFRq4+Cn9OV7/nbFayvk3+M/wSooWaN3bBXZxQlJpl8BqbwrQGOdjVNOwm3r2muKNeZ2MWcddPpHQh0pi2dHKTWJOb7831f+SvvAyJD4rAUROyQ1hJXTL9VF0Z4mfQ4yJK4KYwkLEf4avFkw==


In [42]:
def decrypt_public(file_name, file_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'pkeyutl', '-decrypt', '-inkey', file_key]

    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(file_name, str):
        file_name = base64.b16decode(file_name)
    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=file_name, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout.decode()

In [39]:
c.post('/bin/hackademy/ticket/1511/close')

'STATUS : OK --- ticket 1511 closed\n[ You have new mail ]'

In [94]:
print(c.get('/bin/hackademy/ticket/1512'))

Vous devez programmer la réduction qui sert à établir que déterminer la clef
secrète à partir de la clef publique est aussi dur que factoriser un grand
entier.

Vous trouverez en pièce-jointe de ce ticket une clef publique, et la clef
secrète qui correspond (id="n", "e" et "d"). L'un des deux nombres premiers dont
le produit forme n peut être interprété comme une chaine de caractères. Vous
saurez quoi en faire.

---
Il y a des indications (pj = 'indication-[012]').
[ You have new mail ]


In [113]:
indication0 = c.get('/bin/hackademy/ticket/1512/attachment/indication-0')
indication1 = c.get('/bin/hackademy/ticket/1512/attachment/indication-1')
indication2 = c.get('/bin/hackademy/ticket/1512/attachment/indication-2')

In [114]:
print(indication0)

Comme le déchiffrement RSA est correct, on sait que x**(e*d) = x [modulo n], 
pour n'importe quelle valeur x.

Ceci implique que x**(e*d - 1) = 1 [modulo n].

Vu la façon dont d est fabriqué, on sait que c'est l'inverse de e modulo 
(p-1)*(q-1). Par conséquent, on sait qu'il existe un certain entier k tel que 

     e*d = 1 + k * (p-1)*(q-1).

Comme (p-1) et (q-1) sont pairs, on sait que e*d est impair (il n'y a qu'à
prendre l'égalité ci-dessus modulo 2 des deux côtés pour s'en convaincre).

Par conséquent, x ** ((e*d - 1) // 2) est un nombre dont le carré vaut 1
modulo n. Des nombres comme ça, il y en a 4 : en plus de 1 et -1, il y en a deux
autres qu'on va noter A et -A. Si x est choisi aléatoirement, on a autant de
chance d'obtenir n'importe lequel.

L'important c'est d'obtenir A (ou -A, de toute façon on ne peut pas faire la
différence entre les deux). En effet, on a alors :

    A**2        = 1    [modulo n]
    A**2 - 1    = 0    [modulo n]
    (A-1)*(A+1) = 0    [modulo n]

Ceci

In [97]:
print(indication1)

TL;DR ?

1) Faire : k = (e*d - 1) // 2

3) Faire : x = $$$ et y = x**k modulo n

4) Si y == 1 ou y == n-1, alors retourner à l'étape 3.

5) Renvoyer PGCD(y-1, n). C'est un diviseur de n.


Vu les considérations précédentes, on devrait y arriver après 4 itérations de la
boucle 3--4, en moyenne.


In [27]:

cpt = 0
k = (e*d-1)//2
while True: 
    cpt = cpt + 1
    x = randint(100,1000)
    y = pow(x, k, n)
    print(y)
    if(y!=1) & (y != n-1): 
        resul =  pgcd(y-1, n)
        break
    if (cpt > 20): 
        print("1")
        k = k//2
        cpt = 0

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3736050061638273295195365996933407685763892349757254059419606718259607065371999809919659413586251176926104029053701896868488947637074734013708042735140936729401522082023566038897504548397070386693375030681899612042820423560675135110367593133716611431868446016848341394455179390740415146333316101126117584569504560669129037039836987299564417544933182623067330579398378134199048405304695733599473549156401539881793655335831281759932581324262184619397281688792544135788865040471743347930209462813160024293819218284949845180399443482450677277989357062327344376917658667863386893003928631814093458718267623118508304534224


In [17]:
def pgcd(a,b):
    if b==0:
        return a
    else:
        r=a%b
        return pgcd(b,r)

In [30]:
n = c.get('/bin/hackademy/ticket/1512/attachment/n')
e = c.get('/bin/hackademy/ticket/1512/attachment/e')
d = c.get('/bin/hackademy/ticket/1512/attachment/d')

In [31]:
n =int(n)
e =int(e)
d =int(d)
#reduction = reduction(e, d, n)

print(e)
print(n)
print(d)

65537
1329452130996557008534805442803110177923514555289594756686585243477895213110872438449839983797170697900687339725089037482921112898233397439829988022057174880392669820927259010875169785461598650044800576968566541391820602586892086854654211967005634202322661621989438561707026387196664431139960291011482542807898846838150186454785363015468365105314928281337058286924135230978548331330939486464458088755981999172950821168411186277738539756706808413792531682877344751173870919886010450604569071270952652799140150668411662102476761374250169274227551897869903600560955047687896983580468301849566013606831058459503833733041
12370515869269578939296810277098320776070275510119484787564481333867739732658990042502424873265340723466090201125376456625035522950404968594185319676074681257321175979948137540807422350278347937353255227535151872553825165472001043772300098291337349119740078281300630540003033880712725687745404651793983311031451595628393601123252966507585883845836400599889306617005451118525417

In [32]:
print(resul)
ar = hex(resul)[2:]
ar = unhexlify(ar)
print(ar)

65849758199997936414913711582594585616781985292593180423136384918338569576694324211986751990510123196403039087979070003434458679215808311060246941375469320634876167231438780604196719511220590910486387960197108284146772081739149982204466027280354825651907498463703073532486519701932237343246804600746865732243
b']\xc5\xee\\\x8d-\x9eLC\x03\xf1\xf7#\x8b|\xc7\x85Q&+\xee\xbf\xf0 \x98\xcc\xf4[\x12\x8e\x88\xfc\xb1\x02_\xd1\x03[\x11\xf5\xe5\xa1\x94\xf5b\xa8$\x93\xf5\xd0#\x97c\xb5{-s\xe8\x1e\xaf\xe81\xdd<;\xff\xd5\x98,1\xfb\xdf\xa6\xa8D\x94\xb74\x18\xd4\x11\xdcrXy\xca\x90\x1e\xa8by\xc4\xebo\x1c\\\x8a\xb3\xb9$\xf3y\x85P\xc2}\xdd\x870\xf5\x9a\xe2\xd6\x19\xd0KA\x1bU\xe7\x9a\x05\xa8\xaf\xd6j*\x93'


In [13]:
print(c.get('/bin/hackademy/ticket/1509'))

Vous devez programmer le déchiffrement ElGamal.

Vous trouverez en pièce-jointe (id="p" et "g") des paramètres. Vous devez
génerer une clef publique Elgamal et l'envoyer dans un paramètre "h" à
l'adresse :

    /bin/hackademy/exam/elgamal/decryption

Vous récupérez en échange un message chiffré avec votre clef publique. Vous
devez le déchiffrer. Le nombre obtenu modulo p représente une chaine de 
caractères (cf /doc/ints, "CONVERSION int() <---> bytes()"). Vous saurez quoi en
faire.
[ You have new mail ]


In [55]:
p = c.get('/bin/hackademy/ticket/1508/attachment/p')
g = c.get('/bin/hackademy/ticket/1508/attachment/g')
print("p = {}".format(p))
print("g = {}".format(g))

p = 32317006071311007300338913926423828248817941241140239112842009751400741706634354222619689417363569347117901737909704191754605873209195028853758986185622153212175412514901774520270235796078236248884246189477587641105928646099411723245426622522193230540919037680524235519125679715870117001058055877651038861847280257976054903569732561526167081339361799541336476559160368317896729073178384589680639671900977202194168647225871031411336429319536193471636533209717077448227988588565369208645296636077250268955505928362751121174096972998068410554359584866583291642136218231078990999448652468262416972035911852507045361090559
g = 2


In [73]:
p = int(p)
g = int(g)
a = 5

In [74]:
A = pow(g, a, p)

In [75]:
print(A)

32


In [76]:
reponse = c.post("/bin/hackademy/exam/elgamal/decryption", h= A)

In [77]:
print(reponse)

{'ciphertext': [21579845759817517499155257417495072908797815358037539291771895944455142959328284139191668375885495034703650328786727878841718401554020843677497865158865808015066131805519735679755037677811055062473513484070651991562664957109891689497263387134299903596730117231281049186802100883357377904987850547689715229049220604274449541680903030590234936450151442819949764910220746432121433594547826460487462501135143521828483377340723891339583235253435961259266707225587107629804360639677564956289886025181637871578899719422176107568444094233893252097286017443204746898166354274295231597411485607294714634225349162749743600577031, 76454683893001006576726723813459247719372316635020411206162149047502553110187830741761165512162618637488860182816746772330276036938078297627260760019746624491086638113860344713197681883379152057951514985897620275607014697087820435968664430376814204952638542771625504887272533344123793074042682169227218633757004121054846577168481790055424294122491906630098704979784

In [78]:
message = reponse['ciphertext']
y1 = message[0]
y2 = message[1]

In [79]:
y1 = y1**a
r, y1_pk_inverse, v = euclide_etendu(y1, p)

In [80]:
message = (y1_pk_inverse*y2)%p

In [81]:
message = hex(message)[2:]
message = unhexlify(message)
print(message)

b'Well done! ELGAMAL.DECRYPT:3030:2@71058|3185666b71bff628256d40a92b1893ef'


Remarque : 
- Veuillez trouver dans le script hackademy l'execution des tickets 1507, 1508, 1509, 1512 et 1513 du chapitre "Crypto Hackademy". 
- Dans le fichier banks_dh.ipynb ma connexion à "dh" pour le chapitre "Vous êtes dans la finance ?". 